In [ ]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [1]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# SVC class is too slow to train for large dataset


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

#### Load BERT Vocab Embedding

In [2]:
bertvocab = pd.read_csv('../FMEmbeddings/bertvocab.txt',header=None)
print(bertvocab.shape)
bertvocab = bertvocab.values.tolist()

(65713, 768)


In [3]:
# read counter-fitted vectors
cf = pd.read_table('../FMEmbeddings/counter-fitted-vectors.txt', header=None, sep=' ')
cf.set_index(0, inplace=True)

# check any non-string index
[idx for idx, w in enumerate(cf.index) if type(w) != type('word')]

[6167, 55574]

In [4]:
# cleaned
new_index = list(cf.index)
new_index[6167] = 'null'
new_index[55574] = 'nan'
cf.index = new_index
[idx for idx, w in enumerate(cf.index) if type(w) != type('word')]

[]

In [5]:
# create custom embedding attack

from textattack.shared import WordEmbedding

word2index = dict(zip(new_index, range(65713)))
index2word = dict(zip(range(65713), new_index))
nn_bert = pd.read_csv('../FMEmbeddings/nn_matrix_bert.txt',header=None)
nn_bert = nn_bert.values.tolist()

bertemb = WordEmbedding(bertvocab, word2index, index2word, nn_bert)

In [6]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper, sim=0.5, thre=0.840845057):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## customised wordswapembedding
        transformation = WordSwapEmbedding(50, bertemb)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        
        ### constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        constraints.append(WordEmbeddingDistance(min_cos_sim=sim))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = thre,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [7]:
def runAttack(model_wrapper, n, path, target_ds, sim=0.5, thre=0.840845057):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper, sim, thre)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=50, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [8]:
# classifiers
def adaptation(train_set, test_set):
    lr1 = LogisticRegression(solver='saga', multi_class='multinomial') # default
    lr01 = LogisticRegression(solver='saga', multi_class='multinomial', C=0.1) # strong regularisation
    lr10 = LogisticRegression(solver='saga', multi_class='multinomial', C=10)  # weak regularisation
    
    rf7 = RandomForestClassifier(max_depth=7)
    rf14 = RandomForestClassifier(max_depth=14)
    rf30 = RandomForestClassifier(max_depth=30)
    
    knn7 = KNeighborsClassifier(n_neighbors=7)
    knn35 = KNeighborsClassifier(n_neighbors=35)
    knn175 = KNeighborsClassifier(n_neighbors=175)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]

      
    for key, clf in {'LR':lr1, 'LR C=0.1':lr01, 'LR C=10':lr10,  
                     'RF D=7':rf7, 'RF D=14':rf14, 'RF D=30':rf30,
                     '7-NN':knn7, '35-NN':knn35, '175-NN':knn175}.items():

        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr1, lr01, lr10, rf7, rf14, rf30, knn7, knn35, knn175

### BERT-based-uncased

In [9]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [10]:
bertagtrain = pd.read_csv('../dataset_embeddings/bertagnewstrain.txt', header=None)
bertagtest = pd.read_csv('../dataset_embeddings/bertagnewstest.txt', header=None)
bert_lr1, bert_lr01, bert_lr10, bert_rf7, bert_rf14, bert_rf30, bert_knn7, bert_knn35, bert_knn175 = adaptation(bertagtrain, bertagtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR 0.8942105263157895


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=0.1 0.8864473684210527


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=10 0.8942105263157895
RF D=7 0.7275
RF D=14 0.7975
RF D=30 0.8063157894736842
7-NN 0.7902631578947369
35-NN 0.795
175-NN 0.763421052631579


In [11]:
# custom model wrapper for BERT
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [12]:
bert_lr1_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr1)
bert_lr01_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr01)
bert_lr10_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr10)

bert_rf7_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf7)
bert_rf14_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf14)
bert_rf30_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf30)

bert_knn7_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn7)
bert_knn35_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn35)
bert_knn175_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn175)

### CLIP

In [13]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
clipagtrain = pd.read_csv('../dataset_embeddings/clipagnewstrain.txt', header=None)
clipagtest = pd.read_csv('../dataset_embeddings/clipagnewstest.txt', header=None)
clip_lr1, clip_lr01, clip_lr10, clip_rf7, clip_rf14, clip_rf30, clip_knn7, clip_knn35, clip_knn175 = adaptation(clipagtrain, clipagtest)

LR 0.915921052631579


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=0.1 0.9163157894736842
LR C=10 0.915921052631579
RF D=7 0.8732894736842105
RF D=14 0.9039473684210526
RF D=30 0.9113157894736842
7-NN 0.9248684210526316
35-NN 0.9210526315789473
175-NN 0.9093421052631578


In [15]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [16]:
clip_lr1_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr1)
clip_lr01_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr01)
clip_lr10_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr10)

clip_rf7_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf7)
clip_rf14_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf14)
clip_rf30_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf30)

clip_knn7_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn7)
clip_knn35_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn35)
clip_knn175_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn175)

### Run Attacks

In [17]:
target_ds_ag = textattack.datasets.HuggingFaceDataset('ag_news', split='test')

textattack: Loading datasets dataset ag_news, split test.


In [18]:
n_samples = 300

In [19]:
folder = '../attack_results_new/agbert/'

### Default parameters, threshold = 1 - 0.5/$\pi$ = 0.841, minSim = 0.5

In [20]:
suffix = 'ep0841.csv'
thre = 1-0.5/np.pi

In [21]:
st = datetime.datetime.now()

#### BERT

In [22]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [04:08<20:41,  4.97s/it]textattack: Saving checkpoint under "checkpoints\1723129848746.ta.chkpt" at 2024-08-08 23:10:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 87 / 10 / 100:  33%|███████▎              | 100/300 [06:24<12:48,  3.84s/it]textattack: Saving checkpoint under "checkpoints\1723129984459.ta.chkpt" at 2024-08-08 23:13:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 130 / 15 / 150:  50%|██████████▌          | 150/300 [08:42<08:42,  3.48s/it]textattack: Saving checkpoint under "checkpoints\1723130122937.ta.chkpt" at 2024-08-08 23:15:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 171 / 21 / 200:  67%|██████████████       | 200/300 [11:01<05:30,  3.31s/it]textattack: Saving checkpoint under "checkpoints\1723130262208.ta.chkpt" at 2024-08-08 23:17:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 215 / 25 / 250:  83%|████████████████▋   | 250/300 [13:28<02:41,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1723130408575.ta.chkpt" at 2024-08-08 23:20:08 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [15:52<00:00,  3.17s/it]textattack: Saving checkpoint under "checkpoints\1723130552549.ta.chkpt" at 2024-08-08 23:22:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [15:52<00:00,  3.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.7%   |
| Average perturbed word %:     | 5.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.71  |
+-------------------------------+--------+


In [23]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:13<11:08,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1723130686435.ta.chkpt" at 2024-08-08 23:24:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [04:17<08:34,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723130809812.ta.chkpt" at 2024-08-08 23:26:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 131 / 16 / 150:  50%|██████████▌          | 150/300 [06:29<06:29,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723130942466.ta.chkpt" at 2024-08-08 23:29:02 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 171 / 23 / 200:  67%|██████████████       | 200/300 [08:39<04:19,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723131071963.ta.chkpt" at 2024-08-08 23:31:11 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 216 / 27 / 250:  83%|█████████████████▌   | 250/300 [10:57<02:11,  2.63s/it]textattack: Saving checkpoint under "checkpoints\1723131210507.ta.chkpt" at 2024-08-08 23:33:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:13<00:00,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723131346301.ta.chkpt" at 2024-08-08 23:35:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:13<00:00,  2.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 261    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 87.0%  |
| Attack success rate:          | 2.61%  |
| Average perturbed word %:     | 5.01%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.68  |
+-------------------------------+--------+


In [24]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:03<10:16,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723131469736.ta.chkpt" at 2024-08-08 23:37:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 86 / 11 / 100:  33%|███████▎              | 100/300 [04:03<08:07,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723131590324.ta.chkpt" at 2024-08-08 23:39:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 129 / 16 / 150:  50%|██████████▌          | 150/300 [06:14<06:14,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723131720914.ta.chkpt" at 2024-08-08 23:42:00 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 171 / 22 / 200:  67%|██████████████       | 200/300 [08:27<04:13,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1723131853706.ta.chkpt" at 2024-08-08 23:44:13 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 215 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:43<02:08,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723131989667.ta.chkpt" at 2024-08-08 23:46:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [13:06<00:00,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1723132132875.ta.chkpt" at 2024-08-08 23:48:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [13:06<00:00,  2.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.35%  |
| Average perturbed word %:     | 4.9%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.67  |
+-------------------------------+--------+


In [25]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 27 / 15 / 50:  17%|████                    | 50/300 [01:30<07:30,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723132223059.ta.chkpt" at 2024-08-08 23:50:23 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 57 / 30 / 100:  33%|███████              | 100/300 [03:05<06:11,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723132318804.ta.chkpt" at 2024-08-08 23:51:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 88 / 42 / 150:  50%|██████████▌          | 150/300 [04:59<04:59,  2.00s/it]textattack: Saving checkpoint under "checkpoints\1723132432962.ta.chkpt" at 2024-08-08 23:53:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 118 / 60 / 200:  67%|█████████████▎      | 200/300 [06:49<03:24,  2.05s/it]textattack: Saving checkpoint under "checkpoints\1723132542052.ta.chkpt" at 2024-08-08 23:55:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 152 / 71 / 250:  83%|████████████████▋   | 250/300 [08:44<01:44,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723132658009.ta.chkpt" at 2024-08-08 23:57:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 188 / 84 / 300: 100%|████████████████████| 300/300 [10:42<00:00,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1723132775467.ta.chkpt" at 2024-08-08 23:59:35 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 28 / 188 / 84 / 300: 100%|████████████████████| 300/300 [10:42<00:00,  2.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 28     |
| Number of failed attacks:     | 188    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 62.67% |
| Attack success rate:          | 12.96% |
| Average perturbed word %:     | 4.7%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.33  |
+-------------------------------+--------+


In [26]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 39 / 9 / 50:  17%|████▏                    | 50/300 [01:51<09:18,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723132887297.ta.chkpt" at 2024-08-09 00:01:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 71 / 23 / 100:  33%|███████▎              | 100/300 [03:29<06:59,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723132985388.ta.chkpt" at 2024-08-09 00:03:05 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 71 / 24 / 101:  34%|███████▍              | 101/300 [03:29<06:53,  2.08s/it]

[Succeeded / Failed / Skipped / Total] 9 / 109 / 32 / 150:  50%|██████████▌          | 150/300 [05:27<05:27,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1723133103070.ta.chkpt" at 2024-08-09 00:05:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 143 / 47 / 200:  67%|█████████████▎      | 200/300 [07:22<03:41,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723133218542.ta.chkpt" at 2024-08-09 00:06:58 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 183 / 54 / 250:  83%|████████████████▋   | 250/300 [09:32<01:54,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723133348211.ta.chkpt" at 2024-08-09 00:09:08 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 222 / 61 / 300: 100%|████████████████████| 300/300 [11:43<00:00,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723133479095.ta.chkpt" at 2024-08-09 00:11:19 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 222 / 61 / 300: 100%|████████████████████| 300/300 [11:43<00:00,  2.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 222    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 74.0%  |
| Attack success rate:          | 7.11%  |
| Average perturbed word %:     | 4.45%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.54  |
+-------------------------------+--------+


In [27]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [01:58<09:52,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723133597719.ta.chkpt" at 2024-08-09 00:13:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 75 / 19 / 100:  33%|███████▎              | 100/300 [03:45<07:30,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723133704340.ta.chkpt" at 2024-08-09 00:15:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 112 / 27 / 150:  50%|██████████          | 150/300 [05:45<05:45,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723133825181.ta.chkpt" at 2024-08-09 00:17:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 146 / 39 / 200:  67%|█████████████▎      | 200/300 [07:42<03:51,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723133941692.ta.chkpt" at 2024-08-09 00:19:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 185 / 45 / 250:  83%|████████████████▋   | 250/300 [09:56<01:59,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723134075411.ta.chkpt" at 2024-08-09 00:21:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 223 / 53 / 300: 100%|████████████████████| 300/300 [12:01<00:00,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723134200906.ta.chkpt" at 2024-08-09 00:23:20 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 223 / 53 / 300: 100%|████████████████████| 300/300 [12:01<00:00,  2.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 223    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 74.33% |
| Attack success rate:          | 9.72%  |
| Average perturbed word %:     | 3.72%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.5   |
+-------------------------------+--------+


In [28]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 34 / 11 / 50:  17%|███▋                  | 50/300 [43:07<3:35:39, 51.76s/it]textattack: Saving checkpoint under "checkpoints\1723136789041.ta.chkpt" at 2024-08-09 01:06:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 68 / 20 / 100:  33%|██████▎            | 100/300 [45:25<1:30:50, 27.25s/it]textattack: Saving checkpoint under "checkpoints\1723136926211.ta.chkpt" at 2024-08-09 01:08:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 99 / 32 / 150:  50%|██████████▌          | 150/300 [47:44<47:44, 19.10s/it]textattack: Saving checkpoint under "checkpoints\1723137065960.ta.chkpt" at 2024-08-09 01:11:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 132 / 45 / 200:  67%|█████████████▎      | 200/300 [50:11<25:05, 15.06s/it]textattack: Saving checkpoint under "checkpoints\1723137212506.ta.chkpt" at 2024-08-09 01:13:32 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 168 / 52 / 250:  83%|████████████████▋   | 250/300 [52:41<10:32, 12.65s/it]textattack: Saving checkpoint under "checkpoints\1723137362520.ta.chkpt" at 2024-08-09 01:16:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 207 / 58 / 300: 100%|████████████████████| 300/300 [55:24<00:00, 11.08s/it]textattack: Saving checkpoint under "checkpoints\1723137526006.ta.chkpt" at 2024-08-09 01:18:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 207 / 58 / 300: 100%|████████████████████| 300/300 [55:24<00:00, 11.08s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 35     |
| Number of failed attacks:     | 207    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 69.0%  |
| Attack success rate:          | 14.46% |
| Average perturbed word %:     | 3.55%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.44  |
+-------------------------------+--------+


In [29]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 38 / 9 / 50:  17%|████▏                    | 50/300 [02:20<11:42,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1723137666742.ta.chkpt" at 2024-08-09 01:21:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 73 / 19 / 100:  33%|███████▎              | 100/300 [04:37<09:14,  2.77s/it]textattack: Saving checkpoint under "checkpoints\1723137803195.ta.chkpt" at 2024-08-09 01:23:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 108 / 28 / 150:  50%|██████████          | 150/300 [07:05<07:05,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1723137951630.ta.chkpt" at 2024-08-09 01:25:51 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 141 / 43 / 200:  67%|█████████████▎      | 200/300 [09:29<04:44,  2.85s/it]textattack: Saving checkpoint under "checkpoints\1723138095846.ta.chkpt" at 2024-08-09 01:28:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 180 / 51 / 250:  83%|████████████████▋   | 250/300 [12:05<02:25,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1723138251470.ta.chkpt" at 2024-08-09 01:30:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 219 / 56 / 300: 100%|████████████████████| 300/300 [14:47<00:00,  2.96s/it]textattack: Saving checkpoint under "checkpoints\1723138413359.ta.chkpt" at 2024-08-09 01:33:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 25 / 219 / 56 / 300: 100%|████████████████████| 300/300 [14:47<00:00,  2.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 25     |
| Number of failed attacks:     | 219    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 73.0%  |
| Attack success rate:          | 10.25% |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.44  |
+-------------------------------+--------+


In [30]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 35 / 9 / 50:  17%|████▏                    | 50/300 [02:18<11:32,  2.77s/it]textattack: Saving checkpoint under "checkpoints\1723138552146.ta.chkpt" at 2024-08-09 01:35:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 72 / 20 / 100:  33%|███████▎              | 100/300 [04:37<09:15,  2.78s/it]textattack: Saving checkpoint under "checkpoints\1723138691205.ta.chkpt" at 2024-08-09 01:38:11 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 105 / 33 / 150:  50%|██████████          | 150/300 [06:53<06:53,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1723138826625.ta.chkpt" at 2024-08-09 01:40:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 139 / 47 / 200:  67%|█████████████▎      | 200/300 [09:25<04:42,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1723138979202.ta.chkpt" at 2024-08-09 01:42:59 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 175 / 54 / 250:  83%|████████████████▋   | 250/300 [12:01<02:24,  2.89s/it]textattack: Saving checkpoint under "checkpoints\1723139135562.ta.chkpt" at 2024-08-09 01:45:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 212 / 60 / 300: 100%|████████████████████| 300/300 [14:38<00:00,  2.93s/it]textattack: Saving checkpoint under "checkpoints\1723139292569.ta.chkpt" at 2024-08-09 01:48:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 28 / 212 / 60 / 300: 100%|████████████████████| 300/300 [14:39<00:00,  2.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 28     |
| Number of failed attacks:     | 212    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 70.67% |
| Attack success rate:          | 11.67% |
| Average perturbed word %:     | 3.88%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.28  |
+-------------------------------+--------+


#### CLIP

In [31]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:42<08:31,  2.05s/it]textattack: Saving checkpoint under "checkpoints\1723139395218.ta.chkpt" at 2024-08-09 01:49:55 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:15<06:30,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723139488242.ta.chkpt" at 2024-08-09 01:51:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:54<04:54,  1.96s/it]textattack: Saving checkpoint under "checkpoints\1723139587460.ta.chkpt" at 2024-08-09 01:53:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:29<03:14,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723139682140.ta.chkpt" at 2024-08-09 01:54:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [08:11<01:38,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723139784586.ta.chkpt" at 2024-08-09 01:56:24 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:50<00:00,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723139883547.ta.chkpt" at 2024-08-09 01:58:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:50<00:00,  1.97s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.72  |
+-------------------------------+--------+


In [32]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:30<07:31,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723139973945.ta.chkpt" at 2024-08-09 01:59:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:01<06:03,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723140065156.ta.chkpt" at 2024-08-09 02:01:05 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:38<04:38,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723140162476.ta.chkpt" at 2024-08-09 02:02:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:08<03:04,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723140252251.ta.chkpt" at 2024-08-09 02:04:12 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:49<01:33,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723140353469.ta.chkpt" at 2024-08-09 02:05:53 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:27<00:00,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723140450874.ta.chkpt" at 2024-08-09 02:07:30 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:27<00:00,  1.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.72  |
+-------------------------------+--------+


In [33]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:29<07:28,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723140540720.ta.chkpt" at 2024-08-09 02:09:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:00<06:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723140631289.ta.chkpt" at 2024-08-09 02:10:31 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:38<04:38,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723140728993.ta.chkpt" at 2024-08-09 02:12:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:08<03:04,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723140819375.ta.chkpt" at 2024-08-09 02:13:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:49<01:33,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723140920213.ta.chkpt" at 2024-08-09 02:15:20 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:25<00:00,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723141016361.ta.chkpt" at 2024-08-09 02:16:56 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:25<00:00,  1.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.72  |
+-------------------------------+--------+


In [34]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:30<07:30,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723141106643.ta.chkpt" at 2024-08-09 02:18:26 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 84 / 15 / 100:  33%|███████▎              | 100/300 [02:50<05:41,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723141187400.ta.chkpt" at 2024-08-09 02:19:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 126 / 23 / 150:  50%|██████████▌          | 150/300 [04:21<04:21,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723141277956.ta.chkpt" at 2024-08-09 02:21:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 160 / 37 / 200:  67%|██████████████       | 200/300 [05:30<02:45,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1723141347417.ta.chkpt" at 2024-08-09 02:22:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 200 / 45 / 250:  83%|█████████████████▌   | 250/300 [07:05<01:25,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723141442363.ta.chkpt" at 2024-08-09 02:24:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:40<00:00,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723141536839.ta.chkpt" at 2024-08-09 02:25:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:40<00:00,  1.73s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 244    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 81.33% |
| Attack success rate:          | 2.01%  |
| Average perturbed word %:     | 2.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.44  |
+-------------------------------+--------+


In [35]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:39<08:16,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1723141636189.ta.chkpt" at 2024-08-09 02:27:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [03:09<06:18,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723141726025.ta.chkpt" at 2024-08-09 02:28:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 133 / 14 / 150:  50%|██████████▌          | 150/300 [04:43<04:43,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723141820253.ta.chkpt" at 2024-08-09 02:30:20 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 171 / 25 / 200:  67%|██████████████       | 200/300 [05:56<02:58,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723141893712.ta.chkpt" at 2024-08-09 02:31:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 213 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:34<01:30,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723141991720.ta.chkpt" at 2024-08-09 02:33:11 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:11<00:00,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723142088103.ta.chkpt" at 2024-08-09 02:34:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:11<00:00,  1.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 257    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 85.67% |
| Attack success rate:          | 2.65%  |
| Average perturbed word %:     | 3.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.38  |
+-------------------------------+--------+


In [36]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:38<08:12,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723142186647.ta.chkpt" at 2024-08-09 02:36:26 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 90 / 8 / 100:  33%|███████▋               | 100/300 [03:09<06:18,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723142277686.ta.chkpt" at 2024-08-09 02:37:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 135 / 11 / 150:  50%|██████████▌          | 150/300 [04:45<04:45,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723142374053.ta.chkpt" at 2024-08-09 02:39:34 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 173 / 21 / 200:  67%|██████████████       | 200/300 [06:04<03:02,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723142452528.ta.chkpt" at 2024-08-09 02:40:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 216 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:43<01:32,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723142551740.ta.chkpt" at 2024-08-09 02:42:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 261 / 31 / 300: 100%|█████████████████████| 300/300 [09:22<00:00,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723142650926.ta.chkpt" at 2024-08-09 02:44:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 261 / 31 / 300: 100%|█████████████████████| 300/300 [09:22<00:00,  1.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 261    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 87.0%  |
| Attack success rate:          | 2.97%  |
| Average perturbed word %:     | 2.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.43  |
+-------------------------------+--------+


In [37]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:55<09:39,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723142766884.ta.chkpt" at 2024-08-09 02:46:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [03:56<07:52,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723142887222.ta.chkpt" at 2024-08-09 02:48:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [05:56<05:56,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723143007986.ta.chkpt" at 2024-08-09 02:50:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [07:48<03:54,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723143119763.ta.chkpt" at 2024-08-09 02:51:59 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 231 / 17 / 250:  83%|█████████████████▌   | 250/300 [09:56<01:59,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723143247544.ta.chkpt" at 2024-08-09 02:54:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [11:53<00:00,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723143364228.ta.chkpt" at 2024-08-09 02:56:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [11:53<00:00,  2.38s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 0.72%  |
| Average perturbed word %:     | 3.08%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.44  |
+-------------------------------+--------+


In [38]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [02:01<10:09,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723143486287.ta.chkpt" at 2024-08-09 02:58:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 96 / 3 / 100:  33%|███████▋               | 100/300 [04:03<08:07,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723143608262.ta.chkpt" at 2024-08-09 03:00:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 143 / 6 / 150:  50%|███████████           | 150/300 [06:04<06:04,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723143728882.ta.chkpt" at 2024-08-09 03:02:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 186 / 13 / 200:  67%|██████████████       | 200/300 [07:55<03:57,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723143840018.ta.chkpt" at 2024-08-09 03:04:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 232 / 16 / 250:  83%|█████████████████▌   | 250/300 [10:04<02:00,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723143968752.ta.chkpt" at 2024-08-09 03:06:08 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 275 / 22 / 300: 100%|█████████████████████| 300/300 [11:59<00:00,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1723144083498.ta.chkpt" at 2024-08-09 03:08:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 275 / 22 / 300: 100%|█████████████████████| 300/300 [11:59<00:00,  2.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 275    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 91.67% |
| Attack success rate:          | 1.08%  |
| Average perturbed word %:     | 3.82%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.62  |
+-------------------------------+--------+


In [39]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:56<09:42,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723144200056.ta.chkpt" at 2024-08-09 03:10:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:52<07:45,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723144316255.ta.chkpt" at 2024-08-09 03:11:56 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 139 / 9 / 150:  50%|███████████           | 150/300 [05:56<05:56,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723144439845.ta.chkpt" at 2024-08-09 03:13:59 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 177 / 21 / 200:  67%|██████████████       | 200/300 [07:30<03:45,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723144533650.ta.chkpt" at 2024-08-09 03:15:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 219 / 28 / 250:  83%|█████████████████▌   | 250/300 [09:28<01:53,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723144651706.ta.chkpt" at 2024-08-09 03:17:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:24<00:00,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723144767593.ta.chkpt" at 2024-08-09 03:19:27 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:24<00:00,  2.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 263    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 87.67% |
| Attack success rate:          | 1.13%  |
| Average perturbed word %:     | 4.63%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.27  |
+-------------------------------+--------+


In [40]:
et = datetime.datetime.now()
et - st

datetime.timedelta(seconds=15183, microseconds=364718)

### Threshold = 1 - 0.4/$\pi$ = 0.873

In [41]:
suffix = 'ep0873.csv'
thre = 1-0.4/np.pi

#### BERT

In [42]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:06<10:33,  2.53s/it]textattack: Saving checkpoint under "checkpoints\1723161637805.ta.chkpt" at 2024-08-09 08:00:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 87 / 10 / 100:  33%|███████▎              | 100/300 [04:07<08:14,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1723161758675.ta.chkpt" at 2024-08-09 08:02:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 130 / 15 / 150:  50%|██████████▌          | 150/300 [06:19<06:19,  2.53s/it]textattack: Saving checkpoint under "checkpoints\1723161890586.ta.chkpt" at 2024-08-09 08:04:50 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 171 / 21 / 200:  67%|██████████████       | 200/300 [08:46<04:23,  2.63s/it]textattack: Saving checkpoint under "checkpoints\1723162037908.ta.chkpt" at 2024-08-09 08:07:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 215 / 25 / 250:  83%|████████████████▋   | 250/300 [11:07<02:13,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1723162178863.ta.chkpt" at 2024-08-09 08:09:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [13:23<00:00,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1723162314745.ta.chkpt" at 2024-08-09 08:11:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [13:23<00:00,  2.68s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.7%   |
| Average perturbed word %:     | 4.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.44  |
+-------------------------------+--------+


In [43]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:10<10:51,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723162445681.ta.chkpt" at 2024-08-09 08:14:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [04:13<08:26,  2.53s/it]textattack: Saving checkpoint under "checkpoints\1723162568928.ta.chkpt" at 2024-08-09 08:16:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 131 / 16 / 150:  50%|██████████▌          | 150/300 [06:22<06:22,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723162697709.ta.chkpt" at 2024-08-09 08:18:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 171 / 23 / 200:  67%|██████████████       | 200/300 [08:29<04:14,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723162824845.ta.chkpt" at 2024-08-09 08:20:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 216 / 27 / 250:  83%|█████████████████▌   | 250/300 [10:44<02:08,  2.58s/it]textattack: Saving checkpoint under "checkpoints\1723162960201.ta.chkpt" at 2024-08-09 08:22:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:00<00:00,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723163095542.ta.chkpt" at 2024-08-09 08:24:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:00<00:00,  2.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 261    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 87.0%  |
| Attack success rate:          | 2.61%  |
| Average perturbed word %:     | 5.01%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.42  |
+-------------------------------+--------+


In [44]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [01:59<09:57,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723163215273.ta.chkpt" at 2024-08-09 08:26:55 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 86 / 11 / 100:  33%|███████▎              | 100/300 [03:57<07:55,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723163333689.ta.chkpt" at 2024-08-09 08:28:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 129 / 16 / 150:  50%|██████████▌          | 150/300 [06:08<06:08,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723163464200.ta.chkpt" at 2024-08-09 08:31:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 171 / 22 / 200:  67%|██████████████       | 200/300 [08:19<04:09,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723163595244.ta.chkpt" at 2024-08-09 08:33:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 215 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:36<02:07,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723163732518.ta.chkpt" at 2024-08-09 08:35:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [12:57<00:00,  2.59s/it]textattack: Saving checkpoint under "checkpoints\1723163872875.ta.chkpt" at 2024-08-09 08:37:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [12:57<00:00,  2.59s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.35%  |
| Average perturbed word %:     | 4.9%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.43  |
+-------------------------------+--------+


In [45]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 27 / 15 / 50:  17%|████                    | 50/300 [01:29<07:25,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723163962079.ta.chkpt" at 2024-08-09 08:39:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 57 / 30 / 100:  33%|███████              | 100/300 [03:00<06:01,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723164053729.ta.chkpt" at 2024-08-09 08:40:53 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 57 / 30 / 100:  34%|███████              | 101/300 [03:00<05:56,  1.79s/it]

[Succeeded / Failed / Skipped / Total] 20 / 88 / 42 / 150:  50%|██████████▌          | 150/300 [04:49<04:49,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723164162506.ta.chkpt" at 2024-08-09 08:42:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 118 / 60 / 200:  67%|█████████████▎      | 200/300 [06:34<03:17,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723164267173.ta.chkpt" at 2024-08-09 08:44:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 154 / 71 / 250:  83%|████████████████▋   | 250/300 [08:28<01:41,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1723164381700.ta.chkpt" at 2024-08-09 08:46:21 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 190 / 84 / 300: 100%|████████████████████| 300/300 [10:20<00:00,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1723164493405.ta.chkpt" at 2024-08-09 08:48:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 26 / 190 / 84 / 300: 100%|████████████████████| 300/300 [10:20<00:00,  2.07s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 26     |
| Number of failed attacks:     | 190    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 63.33% |
| Attack success rate:          | 12.04% |
| Average perturbed word %:     | 4.63%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.17  |
+-------------------------------+--------+


In [46]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:49<09:06,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1723164602769.ta.chkpt" at 2024-08-09 08:50:02 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 74 / 23 / 100:  33%|███████▎              | 100/300 [03:27<06:54,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1723164700716.ta.chkpt" at 2024-08-09 08:51:40 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 74 / 24 / 101:  34%|███████▍              | 101/300 [03:27<06:48,  2.05s/it]

[Succeeded / Failed / Skipped / Total] 6 / 112 / 32 / 150:  50%|██████████▌          | 150/300 [05:25<05:25,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723164819321.ta.chkpt" at 2024-08-09 08:53:39 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 146 / 47 / 200:  67%|██████████████       | 200/300 [07:20<03:40,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723164933581.ta.chkpt" at 2024-08-09 08:55:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 186 / 54 / 250:  83%|████████████████▋   | 250/300 [09:27<01:53,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723165060889.ta.chkpt" at 2024-08-09 08:57:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 226 / 61 / 300: 100%|████████████████████| 300/300 [11:33<00:00,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723165187076.ta.chkpt" at 2024-08-09 08:59:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 226 / 61 / 300: 100%|████████████████████| 300/300 [11:33<00:00,  2.31s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 226    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 75.33% |
| Attack success rate:          | 5.44%  |
| Average perturbed word %:     | 4.78%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.31  |
+-------------------------------+--------+


In [47]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [01:59<09:55,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723165306401.ta.chkpt" at 2024-08-09 09:01:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 76 / 19 / 100:  33%|███████▎              | 100/300 [03:44<07:28,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1723165411647.ta.chkpt" at 2024-08-09 09:03:31 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 113 / 27 / 150:  50%|██████████          | 150/300 [05:44<05:44,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723165531472.ta.chkpt" at 2024-08-09 09:05:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 148 / 39 / 200:  67%|█████████████▎      | 200/300 [07:41<03:50,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723165648936.ta.chkpt" at 2024-08-09 09:07:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 188 / 45 / 250:  83%|████████████████▋   | 250/300 [09:53<01:58,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723165780882.ta.chkpt" at 2024-08-09 09:09:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 226 / 53 / 300: 100%|████████████████████| 300/300 [11:58<00:00,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723165905502.ta.chkpt" at 2024-08-09 09:11:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 21 / 226 / 53 / 300: 100%|████████████████████| 300/300 [11:58<00:00,  2.39s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 226    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 75.33% |
| Attack success rate:          | 8.5%   |
| Average perturbed word %:     | 3.69%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.27  |
+-------------------------------+--------+


In [48]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 36 / 11 / 50:  17%|███▋                  | 50/300 [38:51<3:14:17, 46.63s/it]textattack: Saving checkpoint under "checkpoints\1723168237175.ta.chkpt" at 2024-08-09 09:50:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 70 / 20 / 100:  33%|██████▎            | 100/300 [41:09<1:22:19, 24.70s/it]textattack: Saving checkpoint under "checkpoints\1723168375631.ta.chkpt" at 2024-08-09 09:52:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 103 / 32 / 150:  50%|██████████          | 150/300 [43:31<43:31, 17.41s/it]textattack: Saving checkpoint under "checkpoints\1723168516815.ta.chkpt" at 2024-08-09 09:55:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 136 / 45 / 200:  67%|█████████████▎      | 200/300 [46:09<23:04, 13.85s/it]textattack: Saving checkpoint under "checkpoints\1723168675543.ta.chkpt" at 2024-08-09 09:57:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 173 / 52 / 250:  83%|████████████████▋   | 250/300 [48:41<09:44, 11.69s/it]textattack: Saving checkpoint under "checkpoints\1723168827069.ta.chkpt" at 2024-08-09 10:00:27 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 212 / 58 / 300: 100%|████████████████████| 300/300 [51:24<00:00, 10.28s/it]textattack: Saving checkpoint under "checkpoints\1723168990305.ta.chkpt" at 2024-08-09 10:03:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 30 / 212 / 58 / 300: 100%|████████████████████| 300/300 [51:24<00:00, 10.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 30     |
| Number of failed attacks:     | 212    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 70.67% |
| Attack success rate:          | 12.4%  |
| Average perturbed word %:     | 3.63%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.26  |
+-------------------------------+--------+


In [49]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 38 / 9 / 50:  17%|████▏                    | 50/300 [02:26<12:10,  2.92s/it]textattack: Saving checkpoint under "checkpoints\1723169137039.ta.chkpt" at 2024-08-09 10:05:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 74 / 19 / 100:  33%|███████▎              | 100/300 [04:42<09:25,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1723169273445.ta.chkpt" at 2024-08-09 10:07:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 110 / 28 / 150:  50%|██████████          | 150/300 [07:10<07:10,  2.87s/it]textattack: Saving checkpoint under "checkpoints\1723169421783.ta.chkpt" at 2024-08-09 10:10:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 144 / 43 / 200:  67%|█████████████▎      | 200/300 [09:32<04:46,  2.86s/it]textattack: Saving checkpoint under "checkpoints\1723169563233.ta.chkpt" at 2024-08-09 10:12:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 184 / 51 / 250:  83%|████████████████▋   | 250/300 [12:05<02:25,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1723169716561.ta.chkpt" at 2024-08-09 10:15:16 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 224 / 56 / 300: 100%|████████████████████| 300/300 [14:50<00:00,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1723169881032.ta.chkpt" at 2024-08-09 10:18:01 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 20 / 224 / 56 / 300: 100%|████████████████████| 300/300 [14:50<00:00,  2.97s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 20     |
| Number of failed attacks:     | 224    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 74.67% |
| Attack success rate:          | 8.2%   |
| Average perturbed word %:     | 3.62%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.22  |
+-------------------------------+--------+


In [50]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 36 / 9 / 50:  17%|████▏                    | 50/300 [02:14<11:11,  2.69s/it]textattack: Saving checkpoint under "checkpoints\1723170015512.ta.chkpt" at 2024-08-09 10:20:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 73 / 20 / 100:  33%|███████▎              | 100/300 [04:27<08:54,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1723170148165.ta.chkpt" at 2024-08-09 10:22:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 106 / 33 / 150:  50%|██████████          | 150/300 [06:40<06:40,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1723170281679.ta.chkpt" at 2024-08-09 10:24:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 140 / 47 / 200:  67%|█████████████▎      | 200/300 [09:10<04:35,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1723170431254.ta.chkpt" at 2024-08-09 10:27:11 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 177 / 54 / 250:  83%|████████████████▋   | 250/300 [11:44<02:20,  2.82s/it]textattack: Saving checkpoint under "checkpoints\1723170585637.ta.chkpt" at 2024-08-09 10:29:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 216 / 60 / 300: 100%|████████████████████| 300/300 [14:22<00:00,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1723170743745.ta.chkpt" at 2024-08-09 10:32:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 216 / 60 / 300: 100%|████████████████████| 300/300 [14:22<00:00,  2.88s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 24    |
| Number of failed attacks:     | 216   |
| Number of skipped attacks:    | 60    |
| Original accuracy:            | 80.0% |
| Accuracy under attack:        | 72.0% |
| Attack success rate:          | 10.0% |
| Average perturbed word %:     | 3.89% |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 31.1  |
+-------------------------------+-------+


#### CLIP

In [51]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:34<07:51,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723170838192.ta.chkpt" at 2024-08-09 10:33:58 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:04<06:09,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723170928793.ta.chkpt" at 2024-08-09 10:35:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:45<04:45,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723171029397.ta.chkpt" at 2024-08-09 10:37:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:16<03:08,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723171120656.ta.chkpt" at 2024-08-09 10:38:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:59<01:35,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723171223647.ta.chkpt" at 2024-08-09 10:40:23 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:36<00:00,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723171320530.ta.chkpt" at 2024-08-09 10:42:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:36<00:00,  1.92s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 10.2%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.49  |
+-------------------------------+--------+


In [52]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:29<07:25,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723171409632.ta.chkpt" at 2024-08-09 10:43:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:59<05:59,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723171500407.ta.chkpt" at 2024-08-09 10:45:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:35<04:35,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723171595854.ta.chkpt" at 2024-08-09 10:46:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:06<03:03,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723171687220.ta.chkpt" at 2024-08-09 10:48:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:47<01:33,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723171788416.ta.chkpt" at 2024-08-09 10:49:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:25<00:00,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723171886565.ta.chkpt" at 2024-08-09 10:51:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:25<00:00,  1.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 10.2%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.49  |
+-------------------------------+--------+


In [53]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:30<07:30,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723171976868.ta.chkpt" at 2024-08-09 10:52:56 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:02<06:05,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723172069533.ta.chkpt" at 2024-08-09 10:54:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:39<04:39,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723172166510.ta.chkpt" at 2024-08-09 10:56:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:09<03:04,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723172256111.ta.chkpt" at 2024-08-09 10:57:36 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:52<01:34,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723172358725.ta.chkpt" at 2024-08-09 10:59:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723172454686.ta.chkpt" at 2024-08-09 11:00:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 10.2%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.49  |
+-------------------------------+--------+


In [54]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:30<07:31,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723172545254.ta.chkpt" at 2024-08-09 11:02:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 84 / 15 / 100:  33%|███████▎              | 100/300 [02:52<05:44,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723172627292.ta.chkpt" at 2024-08-09 11:03:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 126 / 23 / 150:  50%|██████████▌          | 150/300 [04:23<04:23,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723172718549.ta.chkpt" at 2024-08-09 11:05:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 160 / 37 / 200:  67%|██████████████       | 200/300 [05:33<02:46,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723172788758.ta.chkpt" at 2024-08-09 11:06:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 200 / 45 / 250:  83%|█████████████████▌   | 250/300 [07:09<01:25,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723172884173.ta.chkpt" at 2024-08-09 11:08:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:42<00:00,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723172977587.ta.chkpt" at 2024-08-09 11:09:37 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:42<00:00,  1.74s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 244    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 81.33% |
| Attack success rate:          | 2.01%  |
| Average perturbed word %:     | 2.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.19  |
+-------------------------------+--------+


In [55]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:37<08:06,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1723173074990.ta.chkpt" at 2024-08-09 11:11:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [03:08<06:16,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723173165780.ta.chkpt" at 2024-08-09 11:12:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 133 / 14 / 150:  50%|██████████▌          | 150/300 [04:41<04:41,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723173259412.ta.chkpt" at 2024-08-09 11:14:19 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 171 / 25 / 200:  67%|██████████████       | 200/300 [05:54<02:57,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723173332620.ta.chkpt" at 2024-08-09 11:15:32 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 213 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:33<01:30,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723173431570.ta.chkpt" at 2024-08-09 11:17:11 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:12<00:00,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723173530173.ta.chkpt" at 2024-08-09 11:18:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:12<00:00,  1.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 257    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 85.67% |
| Attack success rate:          | 2.65%  |
| Average perturbed word %:     | 4.32%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.13  |
+-------------------------------+--------+


In [56]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:34<07:52,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723173624852.ta.chkpt" at 2024-08-09 11:20:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 90 / 8 / 100:  33%|███████▋               | 100/300 [03:05<06:10,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723173715796.ta.chkpt" at 2024-08-09 11:21:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 135 / 11 / 150:  50%|██████████▌          | 150/300 [04:42<04:42,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723173813027.ta.chkpt" at 2024-08-09 11:23:33 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 174 / 21 / 200:  67%|██████████████       | 200/300 [06:03<03:01,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723173893705.ta.chkpt" at 2024-08-09 11:24:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 217 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:40<01:32,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723173991221.ta.chkpt" at 2024-08-09 11:26:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 262 / 31 / 300: 100%|█████████████████████| 300/300 [09:19<00:00,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723174090028.ta.chkpt" at 2024-08-09 11:28:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 262 / 31 / 300: 100%|█████████████████████| 300/300 [09:19<00:00,  1.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 2.6%   |
| Average perturbed word %:     | 2.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.19  |
+-------------------------------+--------+


In [57]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [11:09<55:48, 13.39s/it]textattack: Saving checkpoint under "checkpoints\1723174759747.ta.chkpt" at 2024-08-09 11:39:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [13:08<26:17,  7.89s/it]textattack: Saving checkpoint under "checkpoints\1723174878774.ta.chkpt" at 2024-08-09 11:41:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [15:08<15:08,  6.06s/it]textattack: Saving checkpoint under "checkpoints\1723174998836.ta.chkpt" at 2024-08-09 11:43:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [16:59<08:29,  5.10s/it]textattack: Saving checkpoint under "checkpoints\1723175110033.ta.chkpt" at 2024-08-09 11:45:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 231 / 17 / 250:  83%|█████████████████▌   | 250/300 [19:05<03:49,  4.58s/it]textattack: Saving checkpoint under "checkpoints\1723175235965.ta.chkpt" at 2024-08-09 11:47:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [21:03<00:00,  4.21s/it]textattack: Saving checkpoint under "checkpoints\1723175353382.ta.chkpt" at 2024-08-09 11:49:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [21:03<00:00,  4.21s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 0.72%  |
| Average perturbed word %:     | 3.08%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.19  |
+-------------------------------+--------+


In [58]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:58<09:50,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723175471550.ta.chkpt" at 2024-08-09 11:51:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 96 / 3 / 100:  33%|███████▋               | 100/300 [04:01<08:02,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723175594468.ta.chkpt" at 2024-08-09 11:53:14 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 143 / 6 / 150:  50%|███████████           | 150/300 [06:03<06:03,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723175716951.ta.chkpt" at 2024-08-09 11:55:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 186 / 13 / 200:  67%|██████████████       | 200/300 [07:55<03:57,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723175829225.ta.chkpt" at 2024-08-09 11:57:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 233 / 16 / 250:  83%|█████████████████▌   | 250/300 [10:05<02:01,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723175959145.ta.chkpt" at 2024-08-09 11:59:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [12:00<00:00,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1723176074312.ta.chkpt" at 2024-08-09 12:01:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [12:00<00:00,  2.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 0.72%  |
| Average perturbed word %:     | 2.69%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.37  |
+-------------------------------+--------+


In [59]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:55<09:37,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723176189916.ta.chkpt" at 2024-08-09 12:03:09 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:51<07:43,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723176306363.ta.chkpt" at 2024-08-09 12:05:06 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 139 / 9 / 150:  50%|███████████           | 150/300 [05:53<05:53,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723176427724.ta.chkpt" at 2024-08-09 12:07:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 177 / 21 / 200:  67%|██████████████       | 200/300 [07:26<03:43,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723176520435.ta.chkpt" at 2024-08-09 12:08:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 219 / 28 / 250:  83%|█████████████████▌   | 250/300 [09:24<01:52,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723176638501.ta.chkpt" at 2024-08-09 12:10:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:17<00:00,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723176751777.ta.chkpt" at 2024-08-09 12:12:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:17<00:00,  2.26s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 263    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 87.67% |
| Attack success rate:          | 1.13%  |
| Average perturbed word %:     | 4.63%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.02  |
+-------------------------------+--------+


In [60]:
et2 = datetime.datetime.now()
et2 - et

datetime.timedelta(seconds=31984, microseconds=207295)

### Threshold = 0.9

In [61]:
suffix = 'ep09.csv'
thre = 0.9

#### BERT

In [62]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:56<09:43,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723176868791.ta.chkpt" at 2024-08-09 12:14:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 88 / 10 / 100:  33%|███████▎              | 100/300 [03:58<07:57,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723176991036.ta.chkpt" at 2024-08-09 12:16:31 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 131 / 15 / 150:  50%|██████████▌          | 150/300 [06:09<06:09,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1723177121876.ta.chkpt" at 2024-08-09 12:18:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 173 / 21 / 200:  67%|██████████████       | 200/300 [08:37<04:18,  2.59s/it]textattack: Saving checkpoint under "checkpoints\1723177270086.ta.chkpt" at 2024-08-09 12:21:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 218 / 25 / 250:  83%|█████████████████▌   | 250/300 [10:53<02:10,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1723177405661.ta.chkpt" at 2024-08-09 12:23:25 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 263 / 30 / 300: 100%|█████████████████████| 300/300 [13:07<00:00,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1723177539548.ta.chkpt" at 2024-08-09 12:25:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 263 / 30 / 300: 100%|█████████████████████| 300/300 [13:07<00:00,  2.63s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 263    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 87.67% |
| Attack success rate:          | 2.59%  |
| Average perturbed word %:     | 4.21%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.79  |
+-------------------------------+--------+


In [63]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:06<10:30,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723177666194.ta.chkpt" at 2024-08-09 12:27:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 88 / 11 / 100:  33%|███████▎              | 100/300 [04:04<08:08,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723177784501.ta.chkpt" at 2024-08-09 12:29:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 132 / 16 / 150:  50%|██████████▌          | 150/300 [06:11<06:11,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1723177911544.ta.chkpt" at 2024-08-09 12:31:51 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 172 / 23 / 200:  67%|██████████████       | 200/300 [08:17<04:08,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723178037846.ta.chkpt" at 2024-08-09 12:33:57 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 217 / 27 / 250:  83%|█████████████████▌   | 250/300 [10:37<02:07,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723178177125.ta.chkpt" at 2024-08-09 12:36:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 262 / 32 / 300: 100%|█████████████████████| 300/300 [12:50<00:00,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723178310278.ta.chkpt" at 2024-08-09 12:38:30 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 262 / 32 / 300: 100%|█████████████████████| 300/300 [12:50<00:00,  2.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 2.24%  |
| Average perturbed word %:     | 4.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.76  |
+-------------------------------+--------+


In [64]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:58<09:50,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723178428553.ta.chkpt" at 2024-08-09 12:40:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [03:55<07:51,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723178545892.ta.chkpt" at 2024-08-09 12:42:25 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 130 / 16 / 150:  50%|██████████▌          | 150/300 [06:02<06:02,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723178672998.ta.chkpt" at 2024-08-09 12:44:32 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 172 / 22 / 200:  67%|██████████████       | 200/300 [08:10<04:05,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1723178801183.ta.chkpt" at 2024-08-09 12:46:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 217 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:23<02:04,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723178934195.ta.chkpt" at 2024-08-09 12:48:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 262 / 31 / 300: 100%|█████████████████████| 300/300 [12:39<00:00,  2.53s/it]textattack: Saving checkpoint under "checkpoints\1723179070264.ta.chkpt" at 2024-08-09 12:51:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 262 / 31 / 300: 100%|█████████████████████| 300/300 [12:39<00:00,  2.53s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 2.6%   |
| Average perturbed word %:     | 3.78%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.77  |
+-------------------------------+--------+


In [65]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 30 / 15 / 50:  17%|████                    | 50/300 [01:33<07:49,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723179164302.ta.chkpt" at 2024-08-09 12:52:44 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 61 / 30 / 100:  33%|███████▎              | 100/300 [03:07<06:15,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723179257857.ta.chkpt" at 2024-08-09 12:54:17 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 61 / 31 / 101:  34%|███████▍              | 101/300 [03:07<06:09,  1.86s/it]

[Succeeded / Failed / Skipped / Total] 14 / 94 / 42 / 150:  50%|██████████▌          | 150/300 [04:57<04:57,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1723179368169.ta.chkpt" at 2024-08-09 12:56:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 125 / 60 / 200:  67%|█████████████▎      | 200/300 [06:41<03:20,  2.01s/it]textattack: Saving checkpoint under "checkpoints\1723179471804.ta.chkpt" at 2024-08-09 12:57:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 163 / 71 / 250:  83%|████████████████▋   | 250/300 [08:32<01:42,  2.05s/it]textattack: Saving checkpoint under "checkpoints\1723179583125.ta.chkpt" at 2024-08-09 12:59:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 199 / 84 / 300: 100%|████████████████████| 300/300 [10:22<00:00,  2.08s/it]textattack: Saving checkpoint under "checkpoints\1723179692874.ta.chkpt" at 2024-08-09 13:01:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 199 / 84 / 300: 100%|████████████████████| 300/300 [10:22<00:00,  2.08s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 199    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 66.33% |
| Attack success rate:          | 7.87%  |
| Average perturbed word %:     | 4.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.61  |
+-------------------------------+--------+


In [66]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:54<09:30,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723179807190.ta.chkpt" at 2024-08-09 13:03:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 74 / 23 / 100:  33%|███████▎              | 100/300 [03:31<07:03,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723179904542.ta.chkpt" at 2024-08-09 13:05:04 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 74 / 24 / 101:  34%|███████▍              | 101/300 [03:31<06:57,  2.10s/it]

[Succeeded / Failed / Skipped / Total] 6 / 112 / 32 / 150:  50%|██████████▌          | 150/300 [05:28<05:28,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723180021224.ta.chkpt" at 2024-08-09 13:07:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 146 / 47 / 200:  67%|██████████████       | 200/300 [07:19<03:39,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723180132609.ta.chkpt" at 2024-08-09 13:08:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 187 / 54 / 250:  83%|█████████████████▌   | 250/300 [09:26<01:53,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723180259500.ta.chkpt" at 2024-08-09 13:10:59 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 227 / 61 / 300: 100%|████████████████████| 300/300 [11:30<00:00,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723180383573.ta.chkpt" at 2024-08-09 13:13:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 12 / 227 / 61 / 300: 100%|████████████████████| 300/300 [11:30<00:00,  2.30s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 227    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 75.67% |
| Attack success rate:          | 5.02%  |
| Average perturbed word %:     | 4.58%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.72  |
+-------------------------------+--------+


In [67]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [02:01<10:07,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723180505166.ta.chkpt" at 2024-08-09 13:15:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 77 / 19 / 100:  33%|███████▎              | 100/300 [03:46<07:32,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723180609956.ta.chkpt" at 2024-08-09 13:16:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 115 / 27 / 150:  50%|██████████▌          | 150/300 [05:45<05:45,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723180729077.ta.chkpt" at 2024-08-09 13:18:49 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 150 / 39 / 200:  67%|█████████████▎      | 200/300 [07:42<03:51,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723180846042.ta.chkpt" at 2024-08-09 13:20:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 189 / 45 / 250:  83%|████████████████▋   | 250/300 [09:53<01:58,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723180976945.ta.chkpt" at 2024-08-09 13:22:56 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 228 / 53 / 300: 100%|████████████████████| 300/300 [11:56<00:00,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723181100552.ta.chkpt" at 2024-08-09 13:25:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 228 / 53 / 300: 100%|████████████████████| 300/300 [11:56<00:00,  2.39s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 228    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 76.0%  |
| Attack success rate:          | 7.69%  |
| Average perturbed word %:     | 4.59%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.64  |
+-------------------------------+--------+


In [68]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 2 / 37 / 11 / 50:  17%|███▋                  | 50/300 [43:00<3:35:04, 51.62s/it]textattack: Saving checkpoint under "checkpoints\1723183681694.ta.chkpt" at 2024-08-09 14:08:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 71 / 20 / 100:  33%|██████▋             | 100/300 [45:17<1:30:34, 27.17s/it]textattack: Saving checkpoint under "checkpoints\1723183818053.ta.chkpt" at 2024-08-09 14:10:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 105 / 32 / 150:  50%|██████████          | 150/300 [47:35<47:35, 19.03s/it]textattack: Saving checkpoint under "checkpoints\1723183955738.ta.chkpt" at 2024-08-09 14:12:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 140 / 45 / 200:  67%|█████████████▎      | 200/300 [50:17<25:08, 15.09s/it]textattack: Saving checkpoint under "checkpoints\1723184118114.ta.chkpt" at 2024-08-09 14:15:18 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 179 / 52 / 250:  83%|████████████████▋   | 250/300 [52:48<10:33, 12.68s/it]textattack: Saving checkpoint under "checkpoints\1723184269611.ta.chkpt" at 2024-08-09 14:17:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 221 / 58 / 300: 100%|████████████████████| 300/300 [55:24<00:00, 11.08s/it]textattack: Saving checkpoint under "checkpoints\1723184424909.ta.chkpt" at 2024-08-09 14:20:24 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 21 / 221 / 58 / 300: 100%|████████████████████| 300/300 [55:24<00:00, 11.08s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 221    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 73.67% |
| Attack success rate:          | 8.68%  |
| Average perturbed word %:     | 3.88%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.75  |
+-------------------------------+--------+


In [69]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 38 / 9 / 50:  17%|████▏                    | 50/300 [02:19<11:39,  2.80s/it]textattack: Saving checkpoint under "checkpoints\1723184565049.ta.chkpt" at 2024-08-09 14:22:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 75 / 19 / 100:  33%|███████▎              | 100/300 [04:35<09:10,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1723184700548.ta.chkpt" at 2024-08-09 14:25:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 110 / 28 / 150:  50%|██████████          | 150/300 [06:59<06:59,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1723184844267.ta.chkpt" at 2024-08-09 14:27:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 144 / 43 / 200:  67%|█████████████▎      | 200/300 [09:15<04:37,  2.78s/it]textattack: Saving checkpoint under "checkpoints\1723184980358.ta.chkpt" at 2024-08-09 14:29:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 185 / 51 / 250:  83%|████████████████▋   | 250/300 [11:42<02:20,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1723185127530.ta.chkpt" at 2024-08-09 14:32:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 225 / 56 / 300: 100%|████████████████████| 300/300 [14:18<00:00,  2.86s/it]textattack: Saving checkpoint under "checkpoints\1723185283794.ta.chkpt" at 2024-08-09 14:34:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 225 / 56 / 300: 100%|████████████████████| 300/300 [14:18<00:00,  2.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 225    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 75.0%  |
| Attack success rate:          | 7.79%  |
| Average perturbed word %:     | 3.66%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.66  |
+-------------------------------+--------+


In [70]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 4 / 37 / 9 / 50:  17%|████▏                    | 50/300 [02:20<11:43,  2.82s/it]textattack: Saving checkpoint under "checkpoints\1723185424724.ta.chkpt" at 2024-08-09 14:37:04 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 75 / 20 / 100:  33%|███████▎              | 100/300 [04:29<08:59,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1723185553772.ta.chkpt" at 2024-08-09 14:39:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 109 / 33 / 150:  50%|██████████▌          | 150/300 [06:41<06:41,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1723185685206.ta.chkpt" at 2024-08-09 14:41:25 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 145 / 47 / 200:  67%|██████████████       | 200/300 [09:05<04:32,  2.73s/it]textattack: Saving checkpoint under "checkpoints\1723185829144.ta.chkpt" at 2024-08-09 14:43:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 183 / 54 / 250:  83%|████████████████▋   | 250/300 [11:35<02:19,  2.78s/it]textattack: Saving checkpoint under "checkpoints\1723185979467.ta.chkpt" at 2024-08-09 14:46:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 221 / 60 / 300: 100%|████████████████████| 300/300 [14:06<00:00,  2.82s/it]textattack: Saving checkpoint under "checkpoints\1723186130310.ta.chkpt" at 2024-08-09 14:48:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 221 / 60 / 300: 100%|████████████████████| 300/300 [14:06<00:00,  2.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 221    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 73.67% |
| Attack success rate:          | 7.92%  |
| Average perturbed word %:     | 3.61%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.52  |
+-------------------------------+--------+


#### CLIP

In [71]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:37<08:07,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723186228138.ta.chkpt" at 2024-08-09 14:50:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:09<06:19,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723186320459.ta.chkpt" at 2024-08-09 14:52:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:47<04:47,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723186418217.ta.chkpt" at 2024-08-09 14:53:38 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:17<03:08,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723186507788.ta.chkpt" at 2024-08-09 14:55:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:57<01:35,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723186608171.ta.chkpt" at 2024-08-09 14:56:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:33<00:00,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723186703850.ta.chkpt" at 2024-08-09 14:58:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:33<00:00,  1.91s/it]
D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.8   |
+-------------------------------+--------+


In [72]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:28<07:22,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723186792714.ta.chkpt" at 2024-08-09 14:59:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:58<05:56,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723186882488.ta.chkpt" at 2024-08-09 15:01:22 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:31<04:31,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723186976131.ta.chkpt" at 2024-08-09 15:02:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:00<03:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723187064827.ta.chkpt" at 2024-08-09 15:04:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:46<01:33,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723187170289.ta.chkpt" at 2024-08-09 15:06:10 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:22<00:00,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723187266703.ta.chkpt" at 2024-08-09 15:07:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:22<00:00,  1.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.8   |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [73]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:28<07:24,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723187355774.ta.chkpt" at 2024-08-09 15:09:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:00<06:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723187446812.ta.chkpt" at 2024-08-09 15:10:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:36<04:36,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723187543179.ta.chkpt" at 2024-08-09 15:12:23 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:05<03:02,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723187632562.ta.chkpt" at 2024-08-09 15:13:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:47<01:33,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723187733852.ta.chkpt" at 2024-08-09 15:15:33 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:21<00:00,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723187828753.ta.chkpt" at 2024-08-09 15:17:08 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:21<00:00,  1.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.8   |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [74]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:28<07:22,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723187917292.ta.chkpt" at 2024-08-09 15:18:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:46<05:33,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723187995689.ta.chkpt" at 2024-08-09 15:19:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [04:17<04:17,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723188086041.ta.chkpt" at 2024-08-09 15:21:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 162 / 37 / 200:  67%|██████████████       | 200/300 [05:27<02:43,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723188155906.ta.chkpt" at 2024-08-09 15:22:35 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 202 / 45 / 250:  83%|█████████████████▌   | 250/300 [07:00<01:24,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1723188249352.ta.chkpt" at 2024-08-09 15:24:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 246 / 51 / 300: 100%|█████████████████████| 300/300 [08:34<00:00,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723188342968.ta.chkpt" at 2024-08-09 15:25:42 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 246 / 51 / 300: 100%|█████████████████████| 300/300 [08:34<00:00,  1.71s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 3     |
| Number of failed attacks:     | 246   |
| Number of skipped attacks:    | 51    |
| Original accuracy:            | 83.0% |
| Accuracy under attack:        | 82.0% |
| Attack success rate:          | 1.2%  |
| Average perturbed word %:     | 3.75% |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 30.51 |
+-------------------------------+-------+


In [75]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:35<07:58,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723188438885.ta.chkpt" at 2024-08-09 15:27:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [03:05<06:10,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723188528398.ta.chkpt" at 2024-08-09 15:28:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  50%|██████████▌          | 150/300 [04:36<04:36,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723188619452.ta.chkpt" at 2024-08-09 15:30:19 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 173 / 25 / 200:  67%|██████████████       | 200/300 [05:48<02:54,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723188691398.ta.chkpt" at 2024-08-09 15:31:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 217 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:25<01:29,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723188789050.ta.chkpt" at 2024-08-09 15:33:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 261 / 36 / 300: 100%|█████████████████████| 300/300 [09:00<00:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723188883918.ta.chkpt" at 2024-08-09 15:34:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 261 / 36 / 300: 100%|█████████████████████| 300/300 [09:00<00:00,  1.80s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 3     |
| Number of failed attacks:     | 261   |
| Number of skipped attacks:    | 36    |
| Original accuracy:            | 88.0% |
| Accuracy under attack:        | 87.0% |
| Attack success rate:          | 1.14% |
| Average perturbed word %:     | 3.55% |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 30.5  |
+-------------------------------+-------+


In [76]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:45<08:47,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1723188989615.ta.chkpt" at 2024-08-09 15:36:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [03:16<06:32,  1.96s/it]textattack: Saving checkpoint under "checkpoints\1723189080577.ta.chkpt" at 2024-08-09 15:38:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 138 / 11 / 150:  50%|██████████▌          | 150/300 [04:52<04:52,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723189176886.ta.chkpt" at 2024-08-09 15:39:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [06:16<03:08,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723189260393.ta.chkpt" at 2024-08-09 15:41:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 222 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:51<01:34,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723189355823.ta.chkpt" at 2024-08-09 15:42:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723189452254.ta.chkpt" at 2024-08-09 15:44:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 0.74%  |
| Average perturbed word %:     | 2.53%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.54  |
+-------------------------------+--------+


In [77]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [10:32<52:42, 12.65s/it]textattack: Saving checkpoint under "checkpoints\1723190084866.ta.chkpt" at 2024-08-09 15:54:44 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [12:27<24:55,  7.48s/it]textattack: Saving checkpoint under "checkpoints\1723190200018.ta.chkpt" at 2024-08-09 15:56:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [14:23<14:23,  5.76s/it]textattack: Saving checkpoint under "checkpoints\1723190316262.ta.chkpt" at 2024-08-09 15:58:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [16:10<08:05,  4.85s/it]textattack: Saving checkpoint under "checkpoints\1723190422836.ta.chkpt" at 2024-08-09 16:00:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 232 / 17 / 250:  83%|█████████████████▌   | 250/300 [18:12<03:38,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1723190544738.ta.chkpt" at 2024-08-09 16:02:24 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [20:11<00:00,  4.04s/it]textattack: Saving checkpoint under "checkpoints\1723190663623.ta.chkpt" at 2024-08-09 16:04:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [20:11<00:00,  4.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 277    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 3.03%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.53  |
+-------------------------------+--------+


In [78]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:56<09:41,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723190780115.ta.chkpt" at 2024-08-09 16:06:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:56<07:52,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723190900015.ta.chkpt" at 2024-08-09 16:08:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:53<05:53,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723191017350.ta.chkpt" at 2024-08-09 16:10:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [07:40<03:50,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723191124361.ta.chkpt" at 2024-08-09 16:12:04 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [09:44<01:56,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723191247888.ta.chkpt" at 2024-08-09 16:14:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [11:35<00:00,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723191359055.ta.chkpt" at 2024-08-09 16:15:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [11:35<00:00,  2.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 277    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 1.39%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.69  |
+-------------------------------+--------+


In [79]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:52<09:22,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723191471585.ta.chkpt" at 2024-08-09 16:17:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:44<07:28,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1723191583377.ta.chkpt" at 2024-08-09 16:19:43 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 140 / 9 / 150:  50%|███████████           | 150/300 [05:40<05:40,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723191700131.ta.chkpt" at 2024-08-09 16:21:40 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [07:13<03:36,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723191792638.ta.chkpt" at 2024-08-09 16:23:12 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 220 / 28 / 250:  83%|█████████████████▌   | 250/300 [09:07<01:49,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723191906414.ta.chkpt" at 2024-08-09 16:25:06 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 264 / 34 / 300: 100%|█████████████████████| 300/300 [10:56<00:00,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723192016119.ta.chkpt" at 2024-08-09 16:26:56 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 264 / 34 / 300: 100%|█████████████████████| 300/300 [10:56<00:00,  2.19s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 264    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.0%  |
| Attack success rate:          | 0.75%  |
| Average perturbed word %:     | 2.39%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.36  |
+-------------------------------+--------+


In [80]:
et3 = datetime.datetime.now()
et3 - et2

datetime.timedelta(seconds=15264, microseconds=396807)

### Threshold = 0.95

In [81]:
suffix = 'ep095.csv'
thre = 0.95

#### BERT

In [82]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:53<09:28,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723192130848.ta.chkpt" at 2024-08-09 16:28:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 90 / 10 / 100:  33%|███████▎              | 100/300 [03:53<07:46,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723192250311.ta.chkpt" at 2024-08-09 16:30:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 135 / 15 / 150:  50%|██████████▌          | 150/300 [05:55<05:55,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723192372807.ta.chkpt" at 2024-08-09 16:32:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [08:18<04:09,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723192515195.ta.chkpt" at 2024-08-09 16:35:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 225 / 25 / 250:  83%|█████████████████▌   | 250/300 [10:39<02:07,  2.56s/it]textattack: Saving checkpoint under "checkpoints\1723192657156.ta.chkpt" at 2024-08-09 16:37:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 270 / 30 / 300: 100%|█████████████████████| 300/300 [12:49<00:00,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723192786839.ta.chkpt" at 2024-08-09 16:39:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 270 / 30 / 300: 100%|█████████████████████| 300/300 [12:49<00:00,  2.57s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 270   |
| Number of skipped attacks:    | 30    |
| Original accuracy:            | 90.0% |
| Accuracy under attack:        | 90.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 29.2  |
+-------------------------------+-------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [83]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:04<10:24,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723192912750.ta.chkpt" at 2024-08-09 16:41:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [04:00<08:01,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723193028783.ta.chkpt" at 2024-08-09 16:43:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 134 / 16 / 150:  50%|██████████▌          | 150/300 [06:04<06:04,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723193152671.ta.chkpt" at 2024-08-09 16:45:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 177 / 23 / 200:  67%|██████████████       | 200/300 [08:08<04:04,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723193276012.ta.chkpt" at 2024-08-09 16:47:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 223 / 27 / 250:  83%|█████████████████▌   | 250/300 [10:17<02:03,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1723193404979.ta.chkpt" at 2024-08-09 16:50:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [12:26<00:00,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723193534724.ta.chkpt" at 2024-08-09 16:52:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [12:26<00:00,  2.49s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 268    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 89.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.17  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [84]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:03<10:15,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723193657919.ta.chkpt" at 2024-08-09 16:54:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [04:00<08:01,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723193775431.ta.chkpt" at 2024-08-09 16:56:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 134 / 16 / 150:  50%|██████████▌          | 150/300 [06:03<06:03,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723193898387.ta.chkpt" at 2024-08-09 16:58:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 178 / 22 / 200:  67%|██████████████       | 200/300 [08:10<04:05,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1723194025017.ta.chkpt" at 2024-08-09 17:00:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 224 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:20<02:04,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1723194154814.ta.chkpt" at 2024-08-09 17:02:34 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [12:28<00:00,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723194283700.ta.chkpt" at 2024-08-09 17:04:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [12:28<00:00,  2.50s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.18  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [85]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 34 / 15 / 50:  17%|████                    | 50/300 [01:33<07:49,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723194377742.ta.chkpt" at 2024-08-09 17:06:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 69 / 30 / 100:  33%|███████▎              | 100/300 [03:09<06:19,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723194473810.ta.chkpt" at 2024-08-09 17:07:53 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 69 / 31 / 101:  34%|███████▍              | 101/300 [03:10<06:14,  1.88s/it]

[Succeeded / Failed / Skipped / Total] 1 / 107 / 42 / 150:  50%|██████████▌          | 150/300 [04:54<04:54,  1.96s/it]textattack: Saving checkpoint under "checkpoints\1723194577961.ta.chkpt" at 2024-08-09 17:09:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 139 / 60 / 200:  67%|██████████████       | 200/300 [06:35<03:17,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1723194679063.ta.chkpt" at 2024-08-09 17:11:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 71 / 250:  83%|█████████████████▌   | 250/300 [08:20<01:40,  2.00s/it]textattack: Saving checkpoint under "checkpoints\1723194784760.ta.chkpt" at 2024-08-09 17:13:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 215 / 84 / 300: 100%|█████████████████████| 300/300 [10:07<00:00,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723194891115.ta.chkpt" at 2024-08-09 17:14:51 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 215 / 84 / 300: 100%|█████████████████████| 300/300 [10:07<00:00,  2.02s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 215    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 71.67% |
| Attack success rate:          | 0.46%  |
| Average perturbed word %:     | 2.5%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.22  |
+-------------------------------+--------+


In [86]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:51<09:17,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723195002730.ta.chkpt" at 2024-08-09 17:16:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 77 / 23 / 100:  33%|███████▎              | 100/300 [03:27<06:54,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1723195098582.ta.chkpt" at 2024-08-09 17:18:18 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 77 / 24 / 101:  34%|███████▍              | 101/300 [03:27<06:48,  2.05s/it]

[Succeeded / Failed / Skipped / Total] 0 / 118 / 32 / 150:  50%|██████████▌          | 150/300 [05:19<05:19,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723195210867.ta.chkpt" at 2024-08-09 17:20:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 153 / 47 / 200:  67%|██████████████       | 200/300 [07:11<03:35,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723195322753.ta.chkpt" at 2024-08-09 17:22:02 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 196 / 54 / 250:  83%|█████████████████▌   | 250/300 [09:19<01:51,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1723195450936.ta.chkpt" at 2024-08-09 17:24:10 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 239 / 61 / 300: 100%|█████████████████████| 300/300 [11:26<00:00,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723195577840.ta.chkpt" at 2024-08-09 17:26:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 239 / 61 / 300: 100%|█████████████████████| 300/300 [11:26<00:00,  2.29s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 239    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 79.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.28  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [87]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 42 / 8 / 50:  17%|████▏                    | 50/300 [01:56<09:44,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723195694806.ta.chkpt" at 2024-08-09 17:28:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 81 / 19 / 100:  33%|███████▎              | 100/300 [03:40<07:21,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723195798629.ta.chkpt" at 2024-08-09 17:29:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 123 / 27 / 150:  50%|██████████▌          | 150/300 [05:36<05:36,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723195914904.ta.chkpt" at 2024-08-09 17:31:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 161 / 39 / 200:  67%|██████████████       | 200/300 [07:33<03:46,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723196031087.ta.chkpt" at 2024-08-09 17:33:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 205 / 45 / 250:  83%|█████████████████▌   | 250/300 [09:44<01:56,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723196162333.ta.chkpt" at 2024-08-09 17:36:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 247 / 53 / 300: 100%|█████████████████████| 300/300 [11:51<00:00,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723196289454.ta.chkpt" at 2024-08-09 17:38:09 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 247 / 53 / 300: 100%|█████████████████████| 300/300 [11:51<00:00,  2.37s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 247    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 82.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.21  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [88]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 39 / 11 / 50:  17%|███▋                  | 50/300 [29:33<2:27:48, 35.47s/it]textattack: Saving checkpoint under "checkpoints\1723198063210.ta.chkpt" at 2024-08-09 18:07:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 80 / 20 / 100:  33%|██████▋             | 100/300 [31:45<1:03:31, 19.06s/it]textattack: Saving checkpoint under "checkpoints\1723198195133.ta.chkpt" at 2024-08-09 18:09:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 116 / 32 / 150:  50%|██████████▌          | 150/300 [33:48<33:48, 13.52s/it]textattack: Saving checkpoint under "checkpoints\1723198317646.ta.chkpt" at 2024-08-09 18:11:57 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 153 / 45 / 200:  67%|██████████████       | 200/300 [36:00<18:00, 10.80s/it]textattack: Saving checkpoint under "checkpoints\1723198450518.ta.chkpt" at 2024-08-09 18:14:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 196 / 52 / 250:  83%|█████████████████▌   | 250/300 [38:17<07:39,  9.19s/it]textattack: Saving checkpoint under "checkpoints\1723198586990.ta.chkpt" at 2024-08-09 18:16:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 240 / 58 / 300: 100%|█████████████████████| 300/300 [40:43<00:00,  8.15s/it]textattack: Saving checkpoint under "checkpoints\1723198733393.ta.chkpt" at 2024-08-09 18:18:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 240 / 58 / 300: 100%|█████████████████████| 300/300 [40:43<00:00,  8.15s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 240    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 0.83%  |
| Average perturbed word %:     | 2.39%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.26  |
+-------------------------------+--------+


In [89]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:07<10:38,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723198861531.ta.chkpt" at 2024-08-09 18:21:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 81 / 19 / 100:  33%|███████▎              | 100/300 [04:11<08:23,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723198985465.ta.chkpt" at 2024-08-09 18:23:05 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 121 / 28 / 150:  50%|██████████▌          | 150/300 [06:25<06:25,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723199119181.ta.chkpt" at 2024-08-09 18:25:19 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 156 / 43 / 200:  67%|██████████████       | 200/300 [08:33<04:16,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723199247776.ta.chkpt" at 2024-08-09 18:27:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 198 / 51 / 250:  83%|█████████████████▌   | 250/300 [10:46<02:09,  2.59s/it]textattack: Saving checkpoint under "checkpoints\1723199380461.ta.chkpt" at 2024-08-09 18:29:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 243 / 56 / 300: 100%|█████████████████████| 300/300 [13:14<00:00,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723199527932.ta.chkpt" at 2024-08-09 18:32:07 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 243 / 56 / 300: 100%|█████████████████████| 300/300 [13:14<00:00,  2.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 243    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 81.0%  |
| Attack success rate:          | 0.41%  |
| Average perturbed word %:     | 2.7%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.16  |
+-------------------------------+--------+


In [90]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:06<10:31,  2.53s/it]textattack: Saving checkpoint under "checkpoints\1723199654399.ta.chkpt" at 2024-08-09 18:34:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 80 / 20 / 100:  33%|███████▎              | 100/300 [04:11<08:22,  2.51s/it]textattack: Saving checkpoint under "checkpoints\1723199779176.ta.chkpt" at 2024-08-09 18:36:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 117 / 33 / 150:  50%|██████████▌          | 150/300 [06:13<06:13,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723199901124.ta.chkpt" at 2024-08-09 18:38:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 153 / 47 / 200:  67%|██████████████       | 200/300 [08:35<04:17,  2.58s/it]textattack: Saving checkpoint under "checkpoints\1723200043144.ta.chkpt" at 2024-08-09 18:40:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 196 / 54 / 250:  83%|█████████████████▌   | 250/300 [10:55<02:11,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1723200183437.ta.chkpt" at 2024-08-09 18:43:03 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 240 / 60 / 300: 100%|█████████████████████| 300/300 [13:23<00:00,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1723200331347.ta.chkpt" at 2024-08-09 18:45:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 240 / 60 / 300: 100%|█████████████████████| 300/300 [13:23<00:00,  2.68s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 240   |
| Number of skipped attacks:    | 60    |
| Original accuracy:            | 80.0% |
| Accuracy under attack:        | 80.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 29.08 |
+-------------------------------+-------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#### CLIP

In [91]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:45<08:45,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723200436637.ta.chkpt" at 2024-08-09 18:47:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:13<06:26,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723200524890.ta.chkpt" at 2024-08-09 18:48:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:46<04:46,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723200618284.ta.chkpt" at 2024-08-09 18:50:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [06:17<03:08,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723200708450.ta.chkpt" at 2024-08-09 18:51:48 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:58<01:35,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723200810037.ta.chkpt" at 2024-08-09 18:53:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:36<00:00,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723200907754.ta.chkpt" at 2024-08-09 18:55:07 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:36<00:00,  1.92s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.26  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [92]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:27<07:17,  1.75s/it]textattack: Saving checkpoint under "checkpoints\1723200995497.ta.chkpt" at 2024-08-09 18:56:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:55<05:51,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723201083917.ta.chkpt" at 2024-08-09 18:58:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:28<04:28,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723201176782.ta.chkpt" at 2024-08-09 18:59:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:57<02:58,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723201264974.ta.chkpt" at 2024-08-09 19:01:04 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:33<01:30,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723201361444.ta.chkpt" at 2024-08-09 19:02:41 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:05<00:00,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723201453404.ta.chkpt" at 2024-08-09 19:04:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:05<00:00,  1.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.26  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [93]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:25<07:06,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723201538956.ta.chkpt" at 2024-08-09 19:05:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:51<05:43,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723201625388.ta.chkpt" at 2024-08-09 19:07:05 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:23<04:23,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723201717162.ta.chkpt" at 2024-08-09 19:08:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:51<02:55,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723201805648.ta.chkpt" at 2024-08-09 19:10:05 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:28<01:29,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723201902487.ta.chkpt" at 2024-08-09 19:11:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:59<00:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723201993464.ta.chkpt" at 2024-08-09 19:13:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:59<00:00,  1.80s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.26  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [94]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:24<07:02,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1723202078036.ta.chkpt" at 2024-08-09 19:14:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:41<05:22,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723202154699.ta.chkpt" at 2024-08-09 19:15:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [04:06<04:06,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723202239631.ta.chkpt" at 2024-08-09 19:17:19 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 163 / 37 / 200:  67%|██████████████       | 200/300 [05:11<02:35,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1723202305371.ta.chkpt" at 2024-08-09 19:18:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 205 / 45 / 250:  83%|█████████████████▌   | 250/300 [06:45<01:21,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723202399260.ta.chkpt" at 2024-08-09 19:19:59 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 249 / 51 / 300: 100%|█████████████████████| 300/300 [08:20<00:00,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723202494277.ta.chkpt" at 2024-08-09 19:21:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 249 / 51 / 300: 100%|█████████████████████| 300/300 [08:20<00:00,  1.67s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 249   |
| Number of skipped attacks:    | 51    |
| Original accuracy:            | 83.0% |
| Accuracy under attack:        | 83.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 28.99 |
+-------------------------------+-------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [95]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:34<07:52,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723202588862.ta.chkpt" at 2024-08-09 19:23:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [03:03<06:06,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723202677760.ta.chkpt" at 2024-08-09 19:24:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:33<04:33,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723202768059.ta.chkpt" at 2024-08-09 19:26:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 175 / 25 / 200:  67%|██████████████       | 200/300 [05:46<02:53,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723202840920.ta.chkpt" at 2024-08-09 19:27:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 219 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:22<01:28,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723202936739.ta.chkpt" at 2024-08-09 19:28:56 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 264 / 36 / 300: 100%|█████████████████████| 300/300 [08:58<00:00,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723203032829.ta.chkpt" at 2024-08-09 19:30:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 264 / 36 / 300: 100%|█████████████████████| 300/300 [08:58<00:00,  1.79s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 264   |
| Number of skipped attacks:    | 36    |
| Original accuracy:            | 88.0% |
| Accuracy under attack:        | 88.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 28.95 |
+-------------------------------+-------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [96]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:39<08:16,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1723203132163.ta.chkpt" at 2024-08-09 19:32:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [03:08<06:16,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723203221380.ta.chkpt" at 2024-08-09 19:33:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 139 / 11 / 150:  50%|██████████▌          | 150/300 [04:41<04:41,  1.88s/it]textattack: Saving checkpoint under "checkpoints\1723203314766.ta.chkpt" at 2024-08-09 19:35:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [06:08<03:04,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723203401290.ta.chkpt" at 2024-08-09 19:36:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 224 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:42<01:32,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723203495362.ta.chkpt" at 2024-08-09 19:38:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [09:19<00:00,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723203592146.ta.chkpt" at 2024-08-09 19:39:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [09:19<00:00,  1.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 28.97  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [97]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [26:48<2:14:02, 32.17s/it]textattack: Saving checkpoint under "checkpoints\1723205200863.ta.chkpt" at 2024-08-09 20:06:40 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [28:34<57:08, 17.14s/it]textattack: Saving checkpoint under "checkpoints\1723205306561.ta.chkpt" at 2024-08-09 20:08:26 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [30:22<30:22, 12.15s/it]textattack: Saving checkpoint under "checkpoints\1723205414352.ta.chkpt" at 2024-08-09 20:10:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [32:02<16:01,  9.61s/it]textattack: Saving checkpoint under "checkpoints\1723205514404.ta.chkpt" at 2024-08-09 20:11:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 233 / 17 / 250:  83%|█████████████████▌   | 250/300 [33:56<06:47,  8.14s/it]textattack: Saving checkpoint under "checkpoints\1723205628494.ta.chkpt" at 2024-08-09 20:13:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [35:43<00:00,  7.14s/it]textattack: Saving checkpoint under "checkpoints\1723205735693.ta.chkpt" at 2024-08-09 20:15:35 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [35:43<00:00,  7.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 28.96  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [98]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:49<09:09,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723205845706.ta.chkpt" at 2024-08-09 20:17:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:40<07:21,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723205956371.ta.chkpt" at 2024-08-09 20:19:16 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:26<05:26,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723206061961.ta.chkpt" at 2024-08-09 20:21:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [07:06<03:33,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723206162198.ta.chkpt" at 2024-08-09 20:22:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [09:00<01:48,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723206276130.ta.chkpt" at 2024-08-09 20:24:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:50<00:00,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723206386218.ta.chkpt" at 2024-08-09 20:26:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:50<00:00,  2.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.13  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [99]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:46<08:50,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723206492479.ta.chkpt" at 2024-08-09 20:28:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 95 / 5 / 100:  33%|███████▋               | 100/300 [03:32<07:04,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723206598381.ta.chkpt" at 2024-08-09 20:29:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 141 / 9 / 150:  50%|███████████           | 150/300 [05:16<05:16,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1723206703280.ta.chkpt" at 2024-08-09 20:31:43 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [06:40<03:20,  2.00s/it]textattack: Saving checkpoint under "checkpoints\1723206786467.ta.chkpt" at 2024-08-09 20:33:06 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 222 / 28 / 250:  83%|█████████████████▌   | 250/300 [08:25<01:41,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723206891860.ta.chkpt" at 2024-08-09 20:34:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [10:11<00:00,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723206997996.ta.chkpt" at 2024-08-09 20:36:37 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [10:11<00:00,  2.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 28.8   |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [100]:
et4 = datetime.datetime.now()
et4 - et3

datetime.timedelta(seconds=14981, microseconds=892673)

### Threshold = 1 - 0.6/$\pi$ = 0.809

In [101]:
suffix = 'ep0809.csv'
thre = 1-0.6/np.pi

#### BERT

In [102]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:05<10:26,  2.51s/it]textattack: Saving checkpoint under "checkpoints\1723207123961.ta.chkpt" at 2024-08-09 20:38:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 87 / 10 / 100:  33%|███████▎              | 100/300 [04:19<08:39,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723207258249.ta.chkpt" at 2024-08-09 20:40:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 130 / 15 / 150:  50%|██████████▌          | 150/300 [06:38<06:38,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723207396556.ta.chkpt" at 2024-08-09 20:43:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 171 / 21 / 200:  67%|██████████████       | 200/300 [09:11<04:35,  2.76s/it]textattack: Saving checkpoint under "checkpoints\1723207549768.ta.chkpt" at 2024-08-09 20:45:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 215 / 25 / 250:  83%|████████████████▋   | 250/300 [11:38<02:19,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1723207696647.ta.chkpt" at 2024-08-09 20:48:16 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [14:02<00:00,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1723207840976.ta.chkpt" at 2024-08-09 20:50:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 260 / 30 / 300: 100%|████████████████████| 300/300 [14:02<00:00,  2.81s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.7%   |
| Average perturbed word %:     | 5.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.73  |
+-------------------------------+--------+


In [103]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:08<10:43,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723207970040.ta.chkpt" at 2024-08-09 20:52:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [04:17<08:35,  2.58s/it]textattack: Saving checkpoint under "checkpoints\1723208098853.ta.chkpt" at 2024-08-09 20:54:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 131 / 16 / 150:  50%|██████████▌          | 150/300 [06:36<06:36,  2.64s/it]textattack: Saving checkpoint under "checkpoints\1723208237681.ta.chkpt" at 2024-08-09 20:57:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 171 / 23 / 200:  67%|██████████████       | 200/300 [08:49<04:24,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723208371134.ta.chkpt" at 2024-08-09 20:59:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 216 / 27 / 250:  83%|█████████████████▌   | 250/300 [11:16<02:15,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1723208517495.ta.chkpt" at 2024-08-09 21:01:57 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:36<00:00,  2.72s/it]textattack: Saving checkpoint under "checkpoints\1723208657476.ta.chkpt" at 2024-08-09 21:04:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 261 / 32 / 300: 100%|█████████████████████| 300/300 [13:36<00:00,  2.72s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 261    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 87.0%  |
| Attack success rate:          | 2.61%  |
| Average perturbed word %:     | 5.01%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.7   |
+-------------------------------+--------+


In [104]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:04<10:24,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723208782536.ta.chkpt" at 2024-08-09 21:06:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 86 / 11 / 100:  33%|███████▎              | 100/300 [04:10<08:21,  2.51s/it]textattack: Saving checkpoint under "checkpoints\1723208908204.ta.chkpt" at 2024-08-09 21:08:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 129 / 16 / 150:  50%|██████████▌          | 150/300 [06:23<06:23,  2.56s/it]textattack: Saving checkpoint under "checkpoints\1723209041165.ta.chkpt" at 2024-08-09 21:10:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 171 / 22 / 200:  67%|██████████████       | 200/300 [08:41<04:20,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1723209178954.ta.chkpt" at 2024-08-09 21:12:58 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 215 / 26 / 250:  83%|█████████████████▌   | 250/300 [11:01<02:12,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723209319563.ta.chkpt" at 2024-08-09 21:15:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [13:18<00:00,  2.66s/it]textattack: Saving checkpoint under "checkpoints\1723209456472.ta.chkpt" at 2024-08-09 21:17:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 260 / 31 / 300: 100%|█████████████████████| 300/300 [13:18<00:00,  2.66s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 3.35%  |
| Average perturbed word %:     | 4.9%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.69  |
+-------------------------------+--------+


In [105]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 27 / 15 / 50:  17%|████                    | 50/300 [01:31<07:37,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723209548044.ta.chkpt" at 2024-08-09 21:19:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 57 / 30 / 100:  33%|███████              | 100/300 [03:03<06:06,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723209640084.ta.chkpt" at 2024-08-09 21:20:40 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 57 / 31 / 101:  34%|███████              | 101/300 [03:03<06:01,  1.82s/it]

[Succeeded / Failed / Skipped / Total] 20 / 88 / 42 / 150:  50%|██████████▌          | 150/300 [04:51<04:51,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1723209748036.ta.chkpt" at 2024-08-09 21:22:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 118 / 60 / 200:  67%|█████████████▎      | 200/300 [06:35<03:17,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1723209852362.ta.chkpt" at 2024-08-09 21:24:12 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 152 / 71 / 250:  83%|████████████████▋   | 250/300 [08:28<01:41,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1723209965205.ta.chkpt" at 2024-08-09 21:26:05 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 188 / 84 / 300: 100%|████████████████████| 300/300 [10:23<00:00,  2.08s/it]textattack: Saving checkpoint under "checkpoints\1723210079764.ta.chkpt" at 2024-08-09 21:27:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 28 / 188 / 84 / 300: 100%|████████████████████| 300/300 [10:23<00:00,  2.08s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 28     |
| Number of failed attacks:     | 188    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 62.67% |
| Attack success rate:          | 12.96% |
| Average perturbed word %:     | 4.7%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.36  |
+-------------------------------+--------+


In [106]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 39 / 9 / 50:  17%|████▏                    | 50/300 [01:54<09:34,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723210194792.ta.chkpt" at 2024-08-09 21:29:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 71 / 23 / 100:  33%|███████▎              | 100/300 [03:32<07:04,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723210292207.ta.chkpt" at 2024-08-09 21:31:32 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 71 / 24 / 101:  34%|███████▍              | 101/300 [03:32<06:58,  2.10s/it]

[Succeeded / Failed / Skipped / Total] 9 / 109 / 32 / 150:  50%|██████████▌          | 150/300 [05:28<05:28,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723210408250.ta.chkpt" at 2024-08-09 21:33:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 143 / 47 / 200:  67%|█████████████▎      | 200/300 [07:21<03:40,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723210521666.ta.chkpt" at 2024-08-09 21:35:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 183 / 54 / 250:  83%|████████████████▋   | 250/300 [09:33<01:54,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723210653636.ta.chkpt" at 2024-08-09 21:37:33 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 222 / 61 / 300: 100%|████████████████████| 300/300 [11:40<00:00,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723210780539.ta.chkpt" at 2024-08-09 21:39:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 222 / 61 / 300: 100%|████████████████████| 300/300 [11:40<00:00,  2.34s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 222    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 74.0%  |
| Attack success rate:          | 7.11%  |
| Average perturbed word %:     | 4.45%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.57  |
+-------------------------------+--------+


In [107]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [02:04<10:24,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723210905625.ta.chkpt" at 2024-08-09 21:41:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 75 / 19 / 100:  33%|███████▎              | 100/300 [03:53<07:47,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723211014250.ta.chkpt" at 2024-08-09 21:43:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 112 / 27 / 150:  50%|██████████          | 150/300 [05:55<05:55,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723211135747.ta.chkpt" at 2024-08-09 21:45:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 146 / 39 / 200:  67%|█████████████▎      | 200/300 [07:54<03:57,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723211254711.ta.chkpt" at 2024-08-09 21:47:34 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 185 / 45 / 250:  83%|████████████████▋   | 250/300 [10:10<02:02,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723211390802.ta.chkpt" at 2024-08-09 21:49:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 223 / 53 / 300: 100%|████████████████████| 300/300 [12:16<00:00,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723211517597.ta.chkpt" at 2024-08-09 21:51:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 223 / 53 / 300: 100%|████████████████████| 300/300 [12:16<00:00,  2.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 223    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 74.33% |
| Attack success rate:          | 9.72%  |
| Average perturbed word %:     | 3.84%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.53  |
+-------------------------------+--------+


In [108]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 34 / 11 / 50:  17%|███▋                  | 50/300 [27:03<2:15:17, 32.47s/it]textattack: Saving checkpoint under "checkpoints\1723213141166.ta.chkpt" at 2024-08-09 22:19:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 68 / 20 / 100:  33%|███████              | 100/300 [29:22<58:44, 17.62s/it]textattack: Saving checkpoint under "checkpoints\1723213280241.ta.chkpt" at 2024-08-09 22:21:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 99 / 32 / 150:  50%|██████████▌          | 150/300 [31:43<31:43, 12.69s/it]textattack: Saving checkpoint under "checkpoints\1723213421381.ta.chkpt" at 2024-08-09 22:23:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 132 / 45 / 200:  67%|█████████████▎      | 200/300 [34:11<17:05, 10.26s/it]textattack: Saving checkpoint under "checkpoints\1723213568974.ta.chkpt" at 2024-08-09 22:26:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 168 / 52 / 250:  83%|████████████████▋   | 250/300 [36:52<07:22,  8.85s/it]textattack: Saving checkpoint under "checkpoints\1723213730332.ta.chkpt" at 2024-08-09 22:28:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 207 / 58 / 300: 100%|████████████████████| 300/300 [39:39<00:00,  7.93s/it]textattack: Saving checkpoint under "checkpoints\1723213897181.ta.chkpt" at 2024-08-09 22:31:37 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 207 / 58 / 300: 100%|████████████████████| 300/300 [39:39<00:00,  7.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 35     |
| Number of failed attacks:     | 207    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 69.0%  |
| Attack success rate:          | 14.46% |
| Average perturbed word %:     | 3.55%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.46  |
+-------------------------------+--------+


In [109]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 38 / 9 / 50:  17%|████▏                    | 50/300 [02:22<11:51,  2.85s/it]textattack: Saving checkpoint under "checkpoints\1723214039696.ta.chkpt" at 2024-08-09 22:33:59 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 73 / 19 / 100:  33%|███████▎              | 100/300 [04:42<09:25,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1723214180384.ta.chkpt" at 2024-08-09 22:36:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 108 / 28 / 150:  50%|██████████          | 150/300 [07:15<07:15,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1723214333141.ta.chkpt" at 2024-08-09 22:38:53 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 141 / 43 / 200:  67%|█████████████▎      | 200/300 [09:53<04:56,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1723214491022.ta.chkpt" at 2024-08-09 22:41:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 179 / 51 / 250:  83%|████████████████▋   | 250/300 [12:37<02:31,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1723214655364.ta.chkpt" at 2024-08-09 22:44:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 218 / 56 / 300: 100%|████████████████████| 300/300 [15:29<00:00,  3.10s/it]textattack: Saving checkpoint under "checkpoints\1723214826804.ta.chkpt" at 2024-08-09 22:47:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 26 / 218 / 56 / 300: 100%|████████████████████| 300/300 [15:29<00:00,  3.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 26     |
| Number of failed attacks:     | 218    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 72.67% |
| Attack success rate:          | 10.66% |
| Average perturbed word %:     | 3.6%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.45  |
+-------------------------------+--------+


In [110]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 35 / 9 / 50:  17%|████▏                    | 50/300 [02:29<12:29,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1723214977132.ta.chkpt" at 2024-08-09 22:49:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 72 / 20 / 100:  33%|███████▎              | 100/300 [04:49<09:39,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1723215117201.ta.chkpt" at 2024-08-09 22:51:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 105 / 33 / 150:  50%|██████████          | 150/300 [07:10<07:10,  2.87s/it]textattack: Saving checkpoint under "checkpoints\1723215257931.ta.chkpt" at 2024-08-09 22:54:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 139 / 47 / 200:  67%|█████████████▎      | 200/300 [09:46<04:53,  2.93s/it]textattack: Saving checkpoint under "checkpoints\1723215413492.ta.chkpt" at 2024-08-09 22:56:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 175 / 54 / 250:  83%|████████████████▋   | 250/300 [12:27<02:29,  2.99s/it]textattack: Saving checkpoint under "checkpoints\1723215575268.ta.chkpt" at 2024-08-09 22:59:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 212 / 60 / 300: 100%|████████████████████| 300/300 [15:04<00:00,  3.01s/it]textattack: Saving checkpoint under "checkpoints\1723215731584.ta.chkpt" at 2024-08-09 23:02:11 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 28 / 212 / 60 / 300: 100%|████████████████████| 300/300 [15:04<00:00,  3.01s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 28     |
| Number of failed attacks:     | 212    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 70.67% |
| Attack success rate:          | 11.67% |
| Average perturbed word %:     | 3.88%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.3   |
+-------------------------------+--------+


#### CLIP

In [111]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:39<08:18,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1723215831413.ta.chkpt" at 2024-08-09 23:03:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:11<06:22,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723215923033.ta.chkpt" at 2024-08-09 23:05:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:49<04:49,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723216021412.ta.chkpt" at 2024-08-09 23:07:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:19<03:09,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723216111170.ta.chkpt" at 2024-08-09 23:08:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [08:03<01:36,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723216214891.ta.chkpt" at 2024-08-09 23:10:14 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:43<00:00,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723216315539.ta.chkpt" at 2024-08-09 23:11:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:43<00:00,  1.95s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.75  |
+-------------------------------+--------+


In [112]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:34<07:51,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723216410086.ta.chkpt" at 2024-08-09 23:13:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:07<06:14,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723216502915.ta.chkpt" at 2024-08-09 23:15:02 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:46<04:46,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723216602149.ta.chkpt" at 2024-08-09 23:16:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:18<03:09,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723216694083.ta.chkpt" at 2024-08-09 23:18:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [08:01<01:36,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723216797514.ta.chkpt" at 2024-08-09 23:19:57 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:40<00:00,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1723216896499.ta.chkpt" at 2024-08-09 23:21:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:40<00:00,  1.94s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.75  |
+-------------------------------+--------+


In [113]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:32<07:40,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723216988782.ta.chkpt" at 2024-08-09 23:23:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:03<06:07,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723217080523.ta.chkpt" at 2024-08-09 23:24:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:43<04:43,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723217180269.ta.chkpt" at 2024-08-09 23:26:20 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [06:14<03:07,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723217271506.ta.chkpt" at 2024-08-09 23:27:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 223 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:59<01:35,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723217376014.ta.chkpt" at 2024-08-09 23:29:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:37<00:00,  1.92s/it]textattack: Saving checkpoint under "checkpoints\1723217473976.ta.chkpt" at 2024-08-09 23:31:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 267 / 29 / 300: 100%|█████████████████████| 300/300 [09:37<00:00,  1.92s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 5.97%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.75  |
+-------------------------------+--------+


In [114]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:29<07:27,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723217563636.ta.chkpt" at 2024-08-09 23:32:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 84 / 15 / 100:  33%|███████▎              | 100/300 [02:49<05:39,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723217643924.ta.chkpt" at 2024-08-09 23:34:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 126 / 23 / 150:  50%|██████████▌          | 150/300 [04:19<04:19,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723217733824.ta.chkpt" at 2024-08-09 23:35:33 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 160 / 37 / 200:  67%|██████████████       | 200/300 [05:33<02:46,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723217807908.ta.chkpt" at 2024-08-09 23:36:47 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 200 / 45 / 250:  83%|█████████████████▌   | 250/300 [07:09<01:25,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723217903137.ta.chkpt" at 2024-08-09 23:38:23 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:41<00:00,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723217995910.ta.chkpt" at 2024-08-09 23:39:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 244 / 51 / 300: 100%|█████████████████████| 300/300 [08:41<00:00,  1.74s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 244    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 81.33% |
| Attack success rate:          | 2.01%  |
| Average perturbed word %:     | 2.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.46  |
+-------------------------------+--------+


In [115]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:36<08:02,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723218092439.ta.chkpt" at 2024-08-09 23:41:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [03:05<06:11,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723218181729.ta.chkpt" at 2024-08-09 23:43:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 133 / 14 / 150:  50%|██████████▌          | 150/300 [04:39<04:39,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723218275784.ta.chkpt" at 2024-08-09 23:44:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 171 / 25 / 200:  67%|██████████████       | 200/300 [05:53<02:56,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723218349149.ta.chkpt" at 2024-08-09 23:45:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 213 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:32<01:30,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723218448175.ta.chkpt" at 2024-08-09 23:47:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:07<00:00,  1.83s/it]textattack: Saving checkpoint under "checkpoints\1723218543620.ta.chkpt" at 2024-08-09 23:49:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 257 / 36 / 300: 100%|█████████████████████| 300/300 [09:07<00:00,  1.83s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 257    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 85.67% |
| Attack success rate:          | 2.65%  |
| Average perturbed word %:     | 3.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.41  |
+-------------------------------+--------+


In [116]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:41<08:29,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723218645645.ta.chkpt" at 2024-08-09 23:50:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 90 / 8 / 100:  33%|███████▋               | 100/300 [03:14<06:29,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723218738669.ta.chkpt" at 2024-08-09 23:52:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 135 / 11 / 150:  50%|██████████▌          | 150/300 [04:51<04:51,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1723218835427.ta.chkpt" at 2024-08-09 23:53:55 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 173 / 21 / 200:  67%|██████████████       | 200/300 [06:10<03:05,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723218914183.ta.chkpt" at 2024-08-09 23:55:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 216 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:47<01:33,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723219011395.ta.chkpt" at 2024-08-09 23:56:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 261 / 31 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723219112060.ta.chkpt" at 2024-08-09 23:58:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 261 / 31 / 300: 100%|█████████████████████| 300/300 [09:28<00:00,  1.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 261    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 87.0%  |
| Attack success rate:          | 2.97%  |
| Average perturbed word %:     | 2.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.46  |
+-------------------------------+--------+


In [117]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [38:48<3:14:04, 46.58s/it]textattack: Saving checkpoint under "checkpoints\1723221441164.ta.chkpt" at 2024-08-10 00:37:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████              | 100/300 [40:51<1:21:42, 24.51s/it]textattack: Saving checkpoint under "checkpoints\1723221563204.ta.chkpt" at 2024-08-10 00:39:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [42:53<42:53, 17.16s/it]textattack: Saving checkpoint under "checkpoints\1723221685768.ta.chkpt" at 2024-08-10 00:41:25 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [44:49<22:24, 13.45s/it]textattack: Saving checkpoint under "checkpoints\1723221801372.ta.chkpt" at 2024-08-10 00:43:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 231 / 17 / 250:  83%|█████████████████▌   | 250/300 [46:59<09:23, 11.28s/it]textattack: Saving checkpoint under "checkpoints\1723221931858.ta.chkpt" at 2024-08-10 00:45:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [48:57<00:00,  9.79s/it]textattack: Saving checkpoint under "checkpoints\1723222049881.ta.chkpt" at 2024-08-10 00:47:29 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 276 / 22 / 300: 100%|█████████████████████| 300/300 [48:57<00:00,  9.79s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 0.72%  |
| Average perturbed word %:     | 3.08%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.47  |
+-------------------------------+--------+


In [118]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [02:01<10:08,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723222171784.ta.chkpt" at 2024-08-10 00:49:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 96 / 3 / 100:  33%|███████▋               | 100/300 [04:04<08:08,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723222294443.ta.chkpt" at 2024-08-10 00:51:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 143 / 6 / 150:  50%|███████████           | 150/300 [06:05<06:05,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723222415125.ta.chkpt" at 2024-08-10 00:53:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 186 / 13 / 200:  67%|██████████████       | 200/300 [07:58<03:59,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723222528494.ta.chkpt" at 2024-08-10 00:55:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 232 / 16 / 250:  83%|█████████████████▌   | 250/300 [10:08<02:01,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723222658087.ta.chkpt" at 2024-08-10 00:57:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 275 / 22 / 300: 100%|█████████████████████| 300/300 [12:04<00:00,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723222774800.ta.chkpt" at 2024-08-10 00:59:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 275 / 22 / 300: 100%|█████████████████████| 300/300 [12:04<00:00,  2.42s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 275    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 91.67% |
| Attack success rate:          | 1.08%  |
| Average perturbed word %:     | 3.82%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.65  |
+-------------------------------+--------+


In [119]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:56<09:43,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723222891671.ta.chkpt" at 2024-08-10 01:01:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:52<07:45,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723223007479.ta.chkpt" at 2024-08-10 01:03:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 139 / 9 / 150:  50%|███████████           | 150/300 [05:53<05:53,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1723223128038.ta.chkpt" at 2024-08-10 01:05:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 177 / 21 / 200:  67%|██████████████       | 200/300 [07:26<03:43,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723223221532.ta.chkpt" at 2024-08-10 01:07:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 219 / 28 / 250:  83%|█████████████████▌   | 250/300 [09:25<01:53,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723223340387.ta.chkpt" at 2024-08-10 01:09:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:20<00:00,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723223455253.ta.chkpt" at 2024-08-10 01:10:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 263 / 34 / 300: 100%|█████████████████████| 300/300 [11:20<00:00,  2.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 263    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 87.67% |
| Attack success rate:          | 1.13%  |
| Average perturbed word %:     | 4.63%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.3   |
+-------------------------------+--------+


In [120]:
et5 = datetime.datetime.now()
et5 - et4

datetime.timedelta(seconds=16457, microseconds=209714)

### MinSim 0.6, threshold 0.5

In [121]:
suffix = 'sim06.csv'
thre = 1-0.5/np.pi
minsim = 0.6

In [122]:
st = datetime.datetime.now()

#### BERT

In [123]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:54<09:33,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723223570929.ta.chkpt" at 2024-08-10 01:12:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 88 / 10 / 100:  33%|███████▎              | 100/300 [03:57<07:54,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723223693502.ta.chkpt" at 2024-08-10 01:14:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 132 / 15 / 150:  50%|██████████▌          | 150/300 [06:02<06:02,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723223818686.ta.chkpt" at 2024-08-10 01:16:58 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 175 / 21 / 200:  67%|██████████████       | 200/300 [08:31<04:15,  2.56s/it]textattack: Saving checkpoint under "checkpoints\1723223968001.ta.chkpt" at 2024-08-10 01:19:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 221 / 25 / 250:  83%|█████████████████▌   | 250/300 [10:48<02:09,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723224105018.ta.chkpt" at 2024-08-10 01:21:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 266 / 30 / 300: 100%|█████████████████████| 300/300 [12:58<00:00,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723224234843.ta.chkpt" at 2024-08-10 01:23:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 266 / 30 / 300: 100%|█████████████████████| 300/300 [12:58<00:00,  2.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 1.48%  |
| Average perturbed word %:     | 3.4%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.72  |
+-------------------------------+--------+

In [124]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:06<10:31,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723224361669.ta.chkpt" at 2024-08-10 01:26:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [04:01<08:03,  2.42s/it]textattack: Saving checkpoint under "checkpoints\1723224477416.ta.chkpt" at 2024-08-10 01:27:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 132 / 16 / 150:  50%|██████████▌          | 150/300 [06:09<06:09,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723224605083.ta.chkpt" at 2024-08-10 01:30:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 175 / 23 / 200:  67%|██████████████       | 200/300 [08:12<04:06,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723224727975.ta.chkpt" at 2024-08-10 01:32:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 221 / 27 / 250:  83%|█████████████████▌   | 250/300 [10:18<02:03,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1723224854362.ta.chkpt" at 2024-08-10 01:34:14 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 266 / 32 / 300: 100%|█████████████████████| 300/300 [12:30<00:00,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723224986017.ta.chkpt" at 2024-08-10 01:36:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 266 / 32 / 300: 100%|█████████████████████| 300/300 [12:30<00:00,  2.50s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 0.75%  |
| Average perturbed word %:     | 4.47%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.68  |
+-------------------------------+--------+


In [125]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:58<09:52,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723225104738.ta.chkpt" at 2024-08-10 01:38:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 87 / 11 / 100:  33%|███████▎              | 100/300 [03:56<07:52,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1723225222388.ta.chkpt" at 2024-08-10 01:40:22 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 131 / 16 / 150:  50%|██████████▌          | 150/300 [05:59<05:59,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1723225346107.ta.chkpt" at 2024-08-10 01:42:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 175 / 22 / 200:  67%|██████████████       | 200/300 [08:05<04:02,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723225471415.ta.chkpt" at 2024-08-10 01:44:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 221 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:17<02:03,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1723225604007.ta.chkpt" at 2024-08-10 01:46:44 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 266 / 31 / 300: 100%|█████████████████████| 300/300 [12:27<00:00,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723225733349.ta.chkpt" at 2024-08-10 01:48:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 266 / 31 / 300: 100%|█████████████████████| 300/300 [12:27<00:00,  2.49s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 1.12%  |
| Average perturbed word %:     | 2.81%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.7   |
+-------------------------------+--------+


In [126]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 28 / 15 / 50:  17%|████                    | 50/300 [01:33<07:45,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723225826669.ta.chkpt" at 2024-08-10 01:50:26 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 61 / 30 / 100:  33%|███████▎              | 100/300 [03:06<06:13,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723225920009.ta.chkpt" at 2024-08-10 01:52:00 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 61 / 31 / 101:  34%|███████▍              | 101/300 [03:06<06:07,  1.85s/it]

[Succeeded / Failed / Skipped / Total] 13 / 95 / 42 / 150:  50%|██████████▌          | 150/300 [04:52<04:52,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723226025596.ta.chkpt" at 2024-08-10 01:53:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 127 / 60 / 200:  67%|█████████████▎      | 200/300 [06:37<03:18,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1723226130500.ta.chkpt" at 2024-08-10 01:55:30 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 165 / 71 / 250:  83%|████████████████▋   | 250/300 [08:25<01:41,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723226238939.ta.chkpt" at 2024-08-10 01:57:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 202 / 84 / 300: 100%|████████████████████| 300/300 [10:17<00:00,  2.06s/it]textattack: Saving checkpoint under "checkpoints\1723226350888.ta.chkpt" at 2024-08-10 01:59:10 after 300 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 202 / 84 / 300: 100%|████████████████████| 300/300 [10:18<00:00,  2.06s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 202    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 67.33% |
| Attack success rate:          | 6.48%  |
| Average perturbed word %:     | 3.85%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.58  |
+-------------------------------+--------+


In [127]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [01:56<09:40,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723226467958.ta.chkpt" at 2024-08-10 02:01:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 74 / 23 / 100:  33%|███████▎              | 100/300 [03:30<07:00,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723226562139.ta.chkpt" at 2024-08-10 02:02:42 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 74 / 24 / 101:  34%|███████▍              | 101/300 [03:30<06:54,  2.08s/it]

[Succeeded / Failed / Skipped / Total] 3 / 115 / 32 / 150:  50%|██████████▌          | 150/300 [05:24<05:24,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723226675962.ta.chkpt" at 2024-08-10 02:04:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 149 / 47 / 200:  67%|██████████████       | 200/300 [07:12<03:36,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723226784644.ta.chkpt" at 2024-08-10 02:06:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 190 / 54 / 250:  83%|█████████████████▌   | 250/300 [09:15<01:51,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723226906936.ta.chkpt" at 2024-08-10 02:08:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 229 / 61 / 300: 100%|████████████████████| 300/300 [11:19<00:00,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723227031449.ta.chkpt" at 2024-08-10 02:10:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 229 / 61 / 300: 100%|████████████████████| 300/300 [11:19<00:00,  2.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 229    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 76.33% |
| Attack success rate:          | 4.18%  |
| Average perturbed word %:     | 4.02%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.61  |
+-------------------------------+--------+


In [128]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [01:52<09:22,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723227144233.ta.chkpt" at 2024-08-10 02:12:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 78 / 19 / 100:  33%|███████▎              | 100/300 [03:35<07:11,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723227247683.ta.chkpt" at 2024-08-10 02:14:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 116 / 27 / 150:  50%|██████████▌          | 150/300 [05:33<05:33,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723227365010.ta.chkpt" at 2024-08-10 02:16:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 152 / 39 / 200:  67%|██████████████       | 200/300 [07:24<03:42,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723227476553.ta.chkpt" at 2024-08-10 02:17:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 193 / 45 / 250:  83%|████████████████▋   | 250/300 [09:27<01:53,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723227598839.ta.chkpt" at 2024-08-10 02:19:58 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 232 / 53 / 300: 100%|████████████████████| 300/300 [11:25<00:00,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723227717680.ta.chkpt" at 2024-08-10 02:21:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 15 / 232 / 53 / 300: 100%|████████████████████| 300/300 [11:25<00:00,  2.29s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 232    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 77.33% |
| Attack success rate:          | 6.07%  |
| Average perturbed word %:     | 2.96%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.55  |
+-------------------------------+--------+


In [129]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 36 / 11 / 50:  17%|███▋                  | 50/300 [26:54<2:14:33, 32.29s/it]textattack: Saving checkpoint under "checkpoints\1723229332398.ta.chkpt" at 2024-08-10 02:48:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 73 / 20 / 100:  33%|███████▎              | 100/300 [29:06<58:12, 17.46s/it]textattack: Saving checkpoint under "checkpoints\1723229464175.ta.chkpt" at 2024-08-10 02:51:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 108 / 32 / 150:  50%|██████████          | 150/300 [31:19<31:19, 12.53s/it]textattack: Saving checkpoint under "checkpoints\1723229597110.ta.chkpt" at 2024-08-10 02:53:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 142 / 45 / 200:  67%|█████████████▎      | 200/300 [34:07<17:03, 10.24s/it]textattack: Saving checkpoint under "checkpoints\1723229765067.ta.chkpt" at 2024-08-10 02:56:05 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 181 / 52 / 250:  83%|████████████████▋   | 250/300 [36:40<07:20,  8.80s/it]textattack: Saving checkpoint under "checkpoints\1723229918115.ta.chkpt" at 2024-08-10 02:58:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 223 / 58 / 300: 100%|████████████████████| 300/300 [39:16<00:00,  7.85s/it]textattack: Saving checkpoint under "checkpoints\1723230074272.ta.chkpt" at 2024-08-10 03:01:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 223 / 58 / 300: 100%|████████████████████| 300/300 [39:16<00:00,  7.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 223    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 74.33% |
| Attack success rate:          | 7.85%  |
| Average perturbed word %:     | 2.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.62  |
+-------------------------------+--------+


In [130]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 39 / 9 / 50:  17%|████▏                    | 50/300 [02:17<11:25,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1723230211571.ta.chkpt" at 2024-08-10 03:03:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 77 / 19 / 100:  33%|███████▎              | 100/300 [04:28<08:56,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1723230342787.ta.chkpt" at 2024-08-10 03:05:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 115 / 28 / 150:  50%|██████████▌          | 150/300 [06:48<06:48,  2.72s/it]textattack: Saving checkpoint under "checkpoints\1723230482909.ta.chkpt" at 2024-08-10 03:08:02 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 149 / 43 / 200:  67%|██████████████       | 200/300 [09:00<04:30,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1723230614703.ta.chkpt" at 2024-08-10 03:10:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 190 / 51 / 250:  83%|█████████████████▌   | 250/300 [11:37<02:19,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1723230771690.ta.chkpt" at 2024-08-10 03:12:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 230 / 56 / 300: 100%|████████████████████| 300/300 [14:43<00:00,  2.94s/it]textattack: Saving checkpoint under "checkpoints\1723230957567.ta.chkpt" at 2024-08-10 03:15:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 14 / 230 / 56 / 300: 100%|████████████████████| 300/300 [14:43<00:00,  2.94s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 230    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 76.67% |
| Attack success rate:          | 5.74%  |
| Average perturbed word %:     | 2.79%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.58  |
+-------------------------------+--------+


In [131]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 37 / 9 / 50:  17%|████▏                    | 50/300 [02:15<11:18,  2.72s/it]textattack: Saving checkpoint under "checkpoints\1723231093733.ta.chkpt" at 2024-08-10 03:18:13 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 75 / 20 / 100:  33%|███████▎              | 100/300 [04:24<08:48,  2.64s/it]textattack: Saving checkpoint under "checkpoints\1723231222012.ta.chkpt" at 2024-08-10 03:20:22 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 110 / 33 / 150:  50%|██████████▌          | 150/300 [06:32<06:32,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1723231350417.ta.chkpt" at 2024-08-10 03:22:30 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 146 / 47 / 200:  67%|██████████████       | 200/300 [08:51<04:25,  2.66s/it]textattack: Saving checkpoint under "checkpoints\1723231489258.ta.chkpt" at 2024-08-10 03:24:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 187 / 54 / 250:  83%|█████████████████▌   | 250/300 [11:20<02:16,  2.72s/it]textattack: Saving checkpoint under "checkpoints\1723231638769.ta.chkpt" at 2024-08-10 03:27:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 225 / 60 / 300: 100%|████████████████████| 300/300 [13:47<00:00,  2.76s/it]textattack: Saving checkpoint under "checkpoints\1723231785973.ta.chkpt" at 2024-08-10 03:29:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 15 / 225 / 60 / 300: 100%|████████████████████| 300/300 [13:48<00:00,  2.76s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 15    |
| Number of failed attacks:     | 225   |
| Number of skipped attacks:    | 60    |
| Original accuracy:            | 80.0% |
| Accuracy under attack:        | 75.0% |
| Attack success rate:          | 6.25% |
| Average perturbed word %:     | 2.68% |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 30.45 |
+-------------------------------+-------+


#### CLIP

In [132]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:35<07:55,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723231881156.ta.chkpt" at 2024-08-10 03:31:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:02<06:04,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723231968617.ta.chkpt" at 2024-08-10 03:32:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:36<04:36,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723232062452.ta.chkpt" at 2024-08-10 03:34:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:59<02:59,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723232145382.ta.chkpt" at 2024-08-10 03:35:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:40<01:32,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723232246198.ta.chkpt" at 2024-08-10 03:37:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:12<00:00,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723232338365.ta.chkpt" at 2024-08-10 03:38:58 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:12<00:00,  1.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.72  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [133]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:33<07:45,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723232431485.ta.chkpt" at 2024-08-10 03:40:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [03:00<06:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723232518765.ta.chkpt" at 2024-08-10 03:41:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:31<04:31,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723232610132.ta.chkpt" at 2024-08-10 03:43:30 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:55<02:57,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723232693859.ta.chkpt" at 2024-08-10 03:44:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:30<01:30,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723232788927.ta.chkpt" at 2024-08-10 03:46:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:05<00:00,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723232883481.ta.chkpt" at 2024-08-10 03:48:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [09:05<00:00,  1.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.72  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [134]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:25<07:09,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723232969523.ta.chkpt" at 2024-08-10 03:49:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:52<05:44,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723233055696.ta.chkpt" at 2024-08-10 03:50:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:24<04:24,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723233147621.ta.chkpt" at 2024-08-10 03:52:27 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:48<02:54,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723233231956.ta.chkpt" at 2024-08-10 03:53:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:25<01:29,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723233329105.ta.chkpt" at 2024-08-10 03:55:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:58<00:00,  1.80s/it]textattack: Saving checkpoint under "checkpoints\1723233422269.ta.chkpt" at 2024-08-10 03:57:02 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:58<00:00,  1.80s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.72  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [135]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:26<07:12,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723233508854.ta.chkpt" at 2024-08-10 03:58:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:44<05:28,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723233586517.ta.chkpt" at 2024-08-10 03:59:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [04:12<04:12,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1723233675191.ta.chkpt" at 2024-08-10 04:01:15 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 162 / 37 / 200:  67%|██████████████       | 200/300 [05:19<02:39,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723233742321.ta.chkpt" at 2024-08-10 04:02:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 202 / 45 / 250:  83%|█████████████████▌   | 250/300 [06:47<01:21,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1723233829860.ta.chkpt" at 2024-08-10 04:03:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 246 / 51 / 300: 100%|█████████████████████| 300/300 [08:17<00:00,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1723233920374.ta.chkpt" at 2024-08-10 04:05:20 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 246 / 51 / 300: 100%|█████████████████████| 300/300 [08:18<00:00,  1.66s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 3     |
| Number of failed attacks:     | 246   |
| Number of skipped attacks:    | 51    |
| Original accuracy:            | 83.0% |
| Accuracy under attack:        | 82.0% |
| Attack success rate:          | 1.2%  |
| Average perturbed word %:     | 2.8%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 30.43 |
+-------------------------------+-------+


In [136]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:34<07:52,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723234014979.ta.chkpt" at 2024-08-10 04:06:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [02:58<05:56,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723234098533.ta.chkpt" at 2024-08-10 04:08:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 134 / 14 / 150:  50%|██████████▌          | 150/300 [04:25<04:25,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723234185557.ta.chkpt" at 2024-08-10 04:09:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 172 / 25 / 200:  67%|██████████████       | 200/300 [05:31<02:45,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1723234252468.ta.chkpt" at 2024-08-10 04:10:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 215 / 31 / 250:  83%|█████████████████▌   | 250/300 [07:02<01:24,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1723234342866.ta.chkpt" at 2024-08-10 04:12:22 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 259 / 36 / 300: 100%|█████████████████████| 300/300 [08:32<00:00,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723234433140.ta.chkpt" at 2024-08-10 04:13:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 259 / 36 / 300: 100%|█████████████████████| 300/300 [08:32<00:00,  1.71s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 259    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 86.33% |
| Attack success rate:          | 1.89%  |
| Average perturbed word %:     | 3.12%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.37  |
+-------------------------------+--------+


In [137]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:34<07:53,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723234527906.ta.chkpt" at 2024-08-10 04:15:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 90 / 8 / 100:  33%|███████▋               | 100/300 [03:00<06:01,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723234613789.ta.chkpt" at 2024-08-10 04:16:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 136 / 11 / 150:  50%|██████████▌          | 150/300 [04:33<04:33,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723234706250.ta.chkpt" at 2024-08-10 04:18:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 176 / 21 / 200:  67%|██████████████       | 200/300 [05:51<02:55,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723234784897.ta.chkpt" at 2024-08-10 04:19:44 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 220 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:20<01:28,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723234873826.ta.chkpt" at 2024-08-10 04:21:13 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 265 / 31 / 300: 100%|█████████████████████| 300/300 [08:52<00:00,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723234965866.ta.chkpt" at 2024-08-10 04:22:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 265 / 31 / 300: 100%|█████████████████████| 300/300 [08:52<00:00,  1.78s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 265    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 88.33% |
| Attack success rate:          | 1.49%  |
| Average perturbed word %:     | 2.83%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.43  |
+-------------------------------+--------+


In [138]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [34:36<2:53:02, 41.53s/it]textattack: Saving checkpoint under "checkpoints\1723237042444.ta.chkpt" at 2024-08-10 04:57:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████              | 100/300 [36:27<1:12:55, 21.88s/it]textattack: Saving checkpoint under "checkpoints\1723237153766.ta.chkpt" at 2024-08-10 04:59:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [38:20<38:20, 15.33s/it]textattack: Saving checkpoint under "checkpoints\1723237266197.ta.chkpt" at 2024-08-10 05:01:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [40:03<20:01, 12.02s/it]textattack: Saving checkpoint under "checkpoints\1723237369477.ta.chkpt" at 2024-08-10 05:02:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 233 / 17 / 250:  83%|█████████████████▌   | 250/300 [41:59<08:23, 10.08s/it]textattack: Saving checkpoint under "checkpoints\1723237485283.ta.chkpt" at 2024-08-10 05:04:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [43:46<00:00,  8.76s/it]textattack: Saving checkpoint under "checkpoints\1723237592621.ta.chkpt" at 2024-08-10 05:06:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [43:46<00:00,  8.76s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.43  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [139]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:52<09:22,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723237705352.ta.chkpt" at 2024-08-10 05:08:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:46<07:32,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723237818987.ta.chkpt" at 2024-08-10 05:10:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:38<05:38,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723237931499.ta.chkpt" at 2024-08-10 05:12:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [07:20<03:40,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723238032830.ta.chkpt" at 2024-08-10 05:13:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [09:17<01:51,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723238150575.ta.chkpt" at 2024-08-10 05:15:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [11:07<00:00,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723238260349.ta.chkpt" at 2024-08-10 05:17:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [11:07<00:00,  2.23s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 277    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 1.39%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.58  |
+-------------------------------+--------+


In [140]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:49<09:06,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723238369710.ta.chkpt" at 2024-08-10 05:19:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:37<07:14,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723238477829.ta.chkpt" at 2024-08-10 05:21:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 139 / 9 / 150:  50%|███████████           | 150/300 [05:28<05:28,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723238589089.ta.chkpt" at 2024-08-10 05:23:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 177 / 21 / 200:  67%|██████████████       | 200/300 [06:53<03:26,  2.07s/it]textattack: Saving checkpoint under "checkpoints\1723238673909.ta.chkpt" at 2024-08-10 05:24:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 220 / 28 / 250:  83%|█████████████████▌   | 250/300 [08:42<01:44,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723238783112.ta.chkpt" at 2024-08-10 05:26:23 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 264 / 34 / 300: 100%|█████████████████████| 300/300 [10:29<00:00,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723238889450.ta.chkpt" at 2024-08-10 05:28:09 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 264 / 34 / 300: 100%|█████████████████████| 300/300 [10:29<00:00,  2.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 264    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.0%  |
| Attack success rate:          | 0.75%  |
| Average perturbed word %:     | 4.07%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.26  |
+-------------------------------+--------+


In [141]:
et6 = datetime.datetime.now()
et6 - st

datetime.timedelta(seconds=15433, microseconds=579196)

### MinSim 0.7, threshold 0.5

In [142]:
suffix = 'sim07.csv'
thre = 1-0.5/np.pi
minsim = 0.7

#### BERT

In [143]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:50<09:12,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723239000405.ta.chkpt" at 2024-08-10 05:30:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 89 / 10 / 100:  33%|███████▎              | 100/300 [03:45<07:30,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723239114960.ta.chkpt" at 2024-08-10 05:31:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 134 / 15 / 150:  50%|██████████▌          | 150/300 [05:48<05:48,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723239238658.ta.chkpt" at 2024-08-10 05:33:58 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [08:11<04:05,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723239381316.ta.chkpt" at 2024-08-10 05:36:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 224 / 25 / 250:  83%|█████████████████▌   | 250/300 [10:23<02:04,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723239513471.ta.chkpt" at 2024-08-10 05:38:33 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [12:29<00:00,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723239639251.ta.chkpt" at 2024-08-10 05:40:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [12:29<00:00,  2.50s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.11  |
+-------------------------------+--------+


In [144]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:59<09:57,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1723239759822.ta.chkpt" at 2024-08-10 05:42:39 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 88 / 11 / 100:  33%|███████▎              | 100/300 [03:49<07:38,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723239869363.ta.chkpt" at 2024-08-10 05:44:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 133 / 16 / 150:  50%|██████████▌          | 150/300 [05:49<05:49,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1723239989273.ta.chkpt" at 2024-08-10 05:46:29 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 176 / 23 / 200:  67%|██████████████       | 200/300 [07:49<03:54,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1723240109909.ta.chkpt" at 2024-08-10 05:48:29 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 222 / 27 / 250:  83%|█████████████████▌   | 250/300 [09:55<01:59,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1723240235688.ta.chkpt" at 2024-08-10 05:50:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 267 / 32 / 300: 100%|█████████████████████| 300/300 [12:00<00:00,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1723240360465.ta.chkpt" at 2024-08-10 05:52:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 267 / 32 / 300: 100%|█████████████████████| 300/300 [12:00<00:00,  2.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.08  |
+-------------------------------+--------+


In [145]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:50<09:14,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723240471463.ta.chkpt" at 2024-08-10 05:54:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 88 / 11 / 100:  33%|███████▎              | 100/300 [03:43<07:27,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1723240584178.ta.chkpt" at 2024-08-10 05:56:24 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 133 / 16 / 150:  50%|██████████▌          | 150/300 [05:44<05:44,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723240704679.ta.chkpt" at 2024-08-10 05:58:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 177 / 22 / 200:  67%|██████████████       | 200/300 [07:48<03:54,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723240828675.ta.chkpt" at 2024-08-10 06:00:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 223 / 26 / 250:  83%|█████████████████▌   | 250/300 [09:52<01:58,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723240953158.ta.chkpt" at 2024-08-10 06:02:33 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 268 / 31 / 300: 100%|█████████████████████| 300/300 [11:59<00:00,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1723241080062.ta.chkpt" at 2024-08-10 06:04:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 268 / 31 / 300: 100%|█████████████████████| 300/300 [11:59<00:00,  2.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 268    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.33% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.09  |
+-------------------------------+--------+


In [146]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 31 / 15 / 50:  17%|████                    | 50/300 [01:27<07:15,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723241167301.ta.chkpt" at 2024-08-10 06:06:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 64 / 30 / 100:  33%|███████▎              | 100/300 [02:55<05:51,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1723241256106.ta.chkpt" at 2024-08-10 06:07:36 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 64 / 30 / 100:  34%|███████▍              | 101/300 [02:56<05:46,  1.74s/it]

[Succeeded / Failed / Skipped / Total] 9 / 99 / 42 / 150:  50%|███████████           | 150/300 [04:38<04:38,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723241358824.ta.chkpt" at 2024-08-10 06:09:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 131 / 60 / 200:  67%|██████████████       | 200/300 [06:18<03:09,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1723241458365.ta.chkpt" at 2024-08-10 06:10:58 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 169 / 71 / 250:  83%|████████████████▋   | 250/300 [08:01<01:36,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723241562186.ta.chkpt" at 2024-08-10 06:12:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 206 / 84 / 300: 100%|████████████████████| 300/300 [09:45<00:00,  1.95s/it]textattack: Saving checkpoint under "checkpoints\1723241665659.ta.chkpt" at 2024-08-10 06:14:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 206 / 84 / 300: 100%|████████████████████| 300/300 [09:45<00:00,  1.95s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 206    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 68.67% |
| Attack success rate:          | 4.63%  |
| Average perturbed word %:     | 3.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.01  |
+-------------------------------+--------+


In [147]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:50<09:12,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723241776283.ta.chkpt" at 2024-08-10 06:16:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 76 / 23 / 100:  33%|███████▎              | 100/300 [03:22<06:44,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723241867963.ta.chkpt" at 2024-08-10 06:17:47 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 76 / 24 / 101:  34%|███████▍              | 101/300 [03:22<06:38,  2.00s/it]

[Succeeded / Failed / Skipped / Total] 1 / 117 / 32 / 150:  50%|██████████▌          | 150/300 [05:12<05:12,  2.08s/it]textattack: Saving checkpoint under "checkpoints\1723241978426.ta.chkpt" at 2024-08-10 06:19:38 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 152 / 47 / 200:  67%|██████████████       | 200/300 [06:59<03:29,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723242085059.ta.chkpt" at 2024-08-10 06:21:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 193 / 54 / 250:  83%|█████████████████▌   | 250/300 [08:56<01:47,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1723242201978.ta.chkpt" at 2024-08-10 06:23:21 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 235 / 61 / 300: 100%|█████████████████████| 300/300 [10:57<00:00,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723242322928.ta.chkpt" at 2024-08-10 06:25:22 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 235 / 61 / 300: 100%|█████████████████████| 300/300 [10:57<00:00,  2.19s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 235    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 78.33% |
| Attack success rate:          | 1.67%  |
| Average perturbed word %:     | 3.72%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.09  |
+-------------------------------+--------+


In [148]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 41 / 8 / 50:  17%|████▏                    | 50/300 [01:54<09:32,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723242437685.ta.chkpt" at 2024-08-10 06:27:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 78 / 19 / 100:  33%|███████▎              | 100/300 [03:33<07:06,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723242536352.ta.chkpt" at 2024-08-10 06:28:56 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 117 / 27 / 150:  50%|██████████▌          | 150/300 [05:22<05:22,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1723242646049.ta.chkpt" at 2024-08-10 06:30:46 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 153 / 39 / 200:  67%|██████████████       | 200/300 [07:10<03:35,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1723242754108.ta.chkpt" at 2024-08-10 06:32:34 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 194 / 45 / 250:  83%|████████████████▋   | 250/300 [09:19<01:51,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1723242883106.ta.chkpt" at 2024-08-10 06:34:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 234 / 53 / 300: 100%|████████████████████| 300/300 [11:20<00:00,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723243003478.ta.chkpt" at 2024-08-10 06:36:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 234 / 53 / 300: 100%|████████████████████| 300/300 [11:20<00:00,  2.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 234    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 5.26%  |
| Average perturbed word %:     | 3.14%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.01  |
+-------------------------------+--------+


In [149]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 36 / 11 / 50:  17%|███▋                  | 50/300 [23:03<1:55:17, 27.67s/it]textattack: Saving checkpoint under "checkpoints\1723244387096.ta.chkpt" at 2024-08-10 06:59:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 74 / 20 / 100:  33%|███████▎              | 100/300 [25:08<50:17, 15.09s/it]textattack: Saving checkpoint under "checkpoints\1723244512577.ta.chkpt" at 2024-08-10 07:01:52 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 109 / 32 / 150:  50%|██████████▌          | 150/300 [27:15<27:15, 10.91s/it]textattack: Saving checkpoint under "checkpoints\1723244639401.ta.chkpt" at 2024-08-10 07:03:59 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 143 / 45 / 200:  67%|█████████████▎      | 200/300 [29:41<14:50,  8.91s/it]textattack: Saving checkpoint under "checkpoints\1723244785461.ta.chkpt" at 2024-08-10 07:06:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 183 / 52 / 250:  83%|████████████████▋   | 250/300 [31:59<06:23,  7.68s/it]textattack: Saving checkpoint under "checkpoints\1723244922756.ta.chkpt" at 2024-08-10 07:08:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 226 / 58 / 300: 100%|████████████████████| 300/300 [34:27<00:00,  6.89s/it]textattack: Saving checkpoint under "checkpoints\1723245071562.ta.chkpt" at 2024-08-10 07:11:11 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 16 / 226 / 58 / 300: 100%|████████████████████| 300/300 [34:27<00:00,  6.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 226    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 75.33% |
| Attack success rate:          | 6.61%  |
| Average perturbed word %:     | 2.81%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.07  |
+-------------------------------+--------+


In [150]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 39 / 9 / 50:  17%|████▏                    | 50/300 [02:12<11:01,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1723245204864.ta.chkpt" at 2024-08-10 07:13:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 78 / 19 / 100:  33%|███████▎              | 100/300 [04:16<08:33,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723245329079.ta.chkpt" at 2024-08-10 07:15:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 116 / 28 / 150:  50%|██████████▌          | 150/300 [06:30<06:30,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723245462968.ta.chkpt" at 2024-08-10 07:17:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 151 / 43 / 200:  67%|██████████████       | 200/300 [08:39<04:19,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1723245592170.ta.chkpt" at 2024-08-10 07:19:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 192 / 51 / 250:  83%|█████████████████▌   | 250/300 [10:59<02:11,  2.64s/it]textattack: Saving checkpoint under "checkpoints\1723245731638.ta.chkpt" at 2024-08-10 07:22:11 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 235 / 56 / 300: 100%|█████████████████████| 300/300 [13:28<00:00,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1723245881216.ta.chkpt" at 2024-08-10 07:24:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 235 / 56 / 300: 100%|█████████████████████| 300/300 [13:28<00:00,  2.70s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 235    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 78.33% |
| Attack success rate:          | 3.69%  |
| Average perturbed word %:     | 2.9%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.01  |
+-------------------------------+--------+


In [151]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [02:07<10:39,  2.56s/it]textattack: Saving checkpoint under "checkpoints\1723246009229.ta.chkpt" at 2024-08-10 07:26:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 78 / 20 / 100:  33%|███████▎              | 100/300 [04:07<08:14,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1723246128754.ta.chkpt" at 2024-08-10 07:28:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 113 / 33 / 150:  50%|██████████▌          | 150/300 [06:12<06:12,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723246254210.ta.chkpt" at 2024-08-10 07:30:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 149 / 47 / 200:  67%|██████████████       | 200/300 [08:27<04:13,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1723246388758.ta.chkpt" at 2024-08-10 07:33:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 190 / 54 / 250:  83%|█████████████████▌   | 250/300 [10:45<02:09,  2.58s/it]textattack: Saving checkpoint under "checkpoints\1723246526636.ta.chkpt" at 2024-08-10 07:35:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 233 / 60 / 300: 100%|█████████████████████| 300/300 [13:17<00:00,  2.66s/it]textattack: Saving checkpoint under "checkpoints\1723246679232.ta.chkpt" at 2024-08-10 07:37:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 233 / 60 / 300: 100%|█████████████████████| 300/300 [13:17<00:00,  2.66s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 2.92%  |
| Average perturbed word %:     | 3.12%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.91  |
+-------------------------------+--------+


#### CLIP

In [152]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:35<07:57,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723246775070.ta.chkpt" at 2024-08-10 07:39:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:58<05:56,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723246857547.ta.chkpt" at 2024-08-10 07:40:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:27<04:27,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723246946899.ta.chkpt" at 2024-08-10 07:42:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:48<02:54,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723247027906.ta.chkpt" at 2024-08-10 07:43:47 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:18<01:27,  1.75s/it]textattack: Saving checkpoint under "checkpoints\1723247118249.ta.chkpt" at 2024-08-10 07:45:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:46<00:00,  1.75s/it]textattack: Saving checkpoint under "checkpoints\1723247205993.ta.chkpt" at 2024-08-10 07:46:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:46<00:00,  1.76s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.13  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [153]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:21<06:45,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723247287283.ta.chkpt" at 2024-08-10 07:48:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:44<05:28,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723247370369.ta.chkpt" at 2024-08-10 07:49:30 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:12<04:12,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1723247458251.ta.chkpt" at 2024-08-10 07:50:58 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:35<02:47,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1723247541727.ta.chkpt" at 2024-08-10 07:52:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:05<01:25,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723247631897.ta.chkpt" at 2024-08-10 07:53:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:35<00:00,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1723247721951.ta.chkpt" at 2024-08-10 07:55:21 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:35<00:00,  1.72s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.13  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [154]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:21<06:48,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1723247803704.ta.chkpt" at 2024-08-10 07:56:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:45<05:31,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1723247887603.ta.chkpt" at 2024-08-10 07:58:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:13<04:13,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1723247975748.ta.chkpt" at 2024-08-10 07:59:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:34<02:47,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723248056879.ta.chkpt" at 2024-08-10 08:00:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:05<01:25,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723248147669.ta.chkpt" at 2024-08-10 08:02:27 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:32<00:00,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723248234224.ta.chkpt" at 2024-08-10 08:03:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:32<00:00,  1.71s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.13  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [155]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:20<06:44,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723248315268.ta.chkpt" at 2024-08-10 08:05:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:33<05:07,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1723248387947.ta.chkpt" at 2024-08-10 08:06:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [03:57<03:57,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723248471454.ta.chkpt" at 2024-08-10 08:07:51 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 162 / 37 / 200:  67%|██████████████       | 200/300 [04:59<02:29,  1.50s/it]textattack: Saving checkpoint under "checkpoints\1723248534163.ta.chkpt" at 2024-08-10 08:08:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 204 / 45 / 250:  83%|█████████████████▌   | 250/300 [06:24<01:16,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1723248618717.ta.chkpt" at 2024-08-10 08:10:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:47<00:00,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1723248701726.ta.chkpt" at 2024-08-10 08:11:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:47<00:00,  1.56s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 248    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 82.67% |
| Attack success rate:          | 0.4%   |
| Average perturbed word %:     | 1.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.86  |
+-------------------------------+--------+


In [156]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:30<07:32,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1723248792304.ta.chkpt" at 2024-08-10 08:13:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [02:50<05:40,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723248872139.ta.chkpt" at 2024-08-10 08:14:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 134 / 14 / 150:  50%|██████████▌          | 150/300 [04:13<04:13,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1723248955271.ta.chkpt" at 2024-08-10 08:15:55 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 172 / 25 / 200:  67%|██████████████       | 200/300 [05:18<02:39,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723249020555.ta.chkpt" at 2024-08-10 08:17:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 216 / 31 / 250:  83%|█████████████████▌   | 250/300 [06:48<01:21,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1723249110292.ta.chkpt" at 2024-08-10 08:18:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 261 / 36 / 300: 100%|█████████████████████| 300/300 [08:19<00:00,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723249201550.ta.chkpt" at 2024-08-10 08:20:01 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 261 / 36 / 300: 100%|█████████████████████| 300/300 [08:19<00:00,  1.67s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 3     |
| Number of failed attacks:     | 261   |
| Number of skipped attacks:    | 36    |
| Original accuracy:            | 88.0% |
| Accuracy under attack:        | 87.0% |
| Attack success rate:          | 1.14% |
| Average perturbed word %:     | 2.7%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 29.8  |
+-------------------------------+-------+


In [157]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:33<07:48,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723249295394.ta.chkpt" at 2024-08-10 08:21:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  33%|███████▋               | 100/300 [02:58<05:56,  1.78s/it]textattack: Saving checkpoint under "checkpoints\1723249380026.ta.chkpt" at 2024-08-10 08:23:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 138 / 11 / 150:  50%|██████████▌          | 150/300 [04:24<04:24,  1.77s/it]textattack: Saving checkpoint under "checkpoints\1723249466647.ta.chkpt" at 2024-08-10 08:24:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [05:40<02:50,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723249542498.ta.chkpt" at 2024-08-10 08:25:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 222 / 26 / 250:  83%|█████████████████▌   | 250/300 [07:08<01:25,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723249629788.ta.chkpt" at 2024-08-10 08:27:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [08:37<00:00,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723249719311.ta.chkpt" at 2024-08-10 08:28:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [08:37<00:00,  1.73s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 0.74%  |
| Average perturbed word %:     | 2.75%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.85  |
+-------------------------------+--------+


In [158]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [29:08<2:25:44, 34.98s/it]textattack: Saving checkpoint under "checkpoints\1723251468296.ta.chkpt" at 2024-08-10 08:57:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████              | 100/300 [30:54<1:01:49, 18.55s/it]textattack: Saving checkpoint under "checkpoints\1723251574174.ta.chkpt" at 2024-08-10 08:59:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [32:43<32:43, 13.09s/it]textattack: Saving checkpoint under "checkpoints\1723251682624.ta.chkpt" at 2024-08-10 09:01:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [34:24<17:12, 10.32s/it]textattack: Saving checkpoint under "checkpoints\1723251783682.ta.chkpt" at 2024-08-10 09:03:03 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 233 / 17 / 250:  83%|█████████████████▌   | 250/300 [36:18<07:15,  8.72s/it]textattack: Saving checkpoint under "checkpoints\1723251898275.ta.chkpt" at 2024-08-10 09:04:58 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [38:02<00:00,  7.61s/it]textattack: Saving checkpoint under "checkpoints\1723252002078.ta.chkpt" at 2024-08-10 09:06:42 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [38:02<00:00,  7.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.82  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [159]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:50<09:10,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723252112223.ta.chkpt" at 2024-08-10 09:08:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:38<07:17,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723252220845.ta.chkpt" at 2024-08-10 09:10:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:24<05:24,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723252326872.ta.chkpt" at 2024-08-10 09:12:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [07:02<03:31,  2.11s/it]textattack: Saving checkpoint under "checkpoints\1723252424991.ta.chkpt" at 2024-08-10 09:13:44 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [08:53<01:46,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1723252536032.ta.chkpt" at 2024-08-10 09:15:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:37<00:00,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723252639605.ta.chkpt" at 2024-08-10 09:17:19 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:37<00:00,  2.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.98  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [160]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:42<08:30,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723252741715.ta.chkpt" at 2024-08-10 09:19:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 94 / 5 / 100:  33%|███████▋               | 100/300 [03:24<06:49,  2.05s/it]textattack: Saving checkpoint under "checkpoints\1723252844241.ta.chkpt" at 2024-08-10 09:20:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 140 / 9 / 150:  50%|███████████           | 150/300 [05:11<05:11,  2.08s/it]textattack: Saving checkpoint under "checkpoints\1723252951518.ta.chkpt" at 2024-08-10 09:22:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [06:34<03:17,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723253033729.ta.chkpt" at 2024-08-10 09:23:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 221 / 28 / 250:  83%|█████████████████▌   | 250/300 [08:16<01:39,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1723253136129.ta.chkpt" at 2024-08-10 09:25:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 265 / 34 / 300: 100%|█████████████████████| 300/300 [09:56<00:00,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1723253236491.ta.chkpt" at 2024-08-10 09:27:16 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 265 / 34 / 300: 100%|█████████████████████| 300/300 [09:56<00:00,  1.99s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 265    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.33% |
| Attack success rate:          | 0.38%  |
| Average perturbed word %:     | 2.08%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.67  |
+-------------------------------+--------+


In [161]:
et7 = datetime.datetime.now()
et7 - et6

datetime.timedelta(seconds=14347, microseconds=57061)

### MinSim 0.8, threshold 0.5

In [162]:
suffix = 'sim08.csv'
thre = 1-0.5/np.pi
minsim = 0.8

#### BERT

In [163]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:45<08:49,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723253342974.ta.chkpt" at 2024-08-10 09:29:02 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 89 / 10 / 100:  33%|███████▎              | 100/300 [03:36<07:12,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1723253453484.ta.chkpt" at 2024-08-10 09:30:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 134 / 15 / 150:  50%|██████████▌          | 150/300 [05:29<05:29,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723253566800.ta.chkpt" at 2024-08-10 09:32:46 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [07:25<03:42,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723253682336.ta.chkpt" at 2024-08-10 09:34:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 224 / 25 / 250:  83%|█████████████████▌   | 250/300 [09:29<01:53,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723253806677.ta.chkpt" at 2024-08-10 09:36:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [11:33<00:00,  2.31s/it]textattack: Saving checkpoint under "checkpoints\1723253930846.ta.chkpt" at 2024-08-10 09:38:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [11:33<00:00,  2.31s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.61  |
+-------------------------------+--------+


In [164]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:51<09:17,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723254042562.ta.chkpt" at 2024-08-10 09:40:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 88 / 11 / 100:  33%|███████▎              | 100/300 [03:38<07:17,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723254149979.ta.chkpt" at 2024-08-10 09:42:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 133 / 16 / 150:  50%|██████████▌          | 150/300 [05:32<05:32,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723254263523.ta.chkpt" at 2024-08-10 09:44:23 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 176 / 23 / 200:  67%|██████████████       | 200/300 [07:26<03:43,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723254377987.ta.chkpt" at 2024-08-10 09:46:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 222 / 27 / 250:  83%|█████████████████▌   | 250/300 [09:27<01:53,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1723254498611.ta.chkpt" at 2024-08-10 09:48:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 267 / 32 / 300: 100%|█████████████████████| 300/300 [11:29<00:00,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723254621028.ta.chkpt" at 2024-08-10 09:50:21 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 267 / 32 / 300: 100%|█████████████████████| 300/300 [11:29<00:00,  2.30s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.58  |
+-------------------------------+--------+


In [165]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:50<09:11,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723254731401.ta.chkpt" at 2024-08-10 09:52:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 88 / 11 / 100:  33%|███████▎              | 100/300 [03:49<07:38,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723254850522.ta.chkpt" at 2024-08-10 09:54:10 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 133 / 16 / 150:  50%|██████████▌          | 150/300 [05:48<05:48,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723254969530.ta.chkpt" at 2024-08-10 09:56:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 177 / 22 / 200:  67%|██████████████       | 200/300 [08:07<04:03,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723255109093.ta.chkpt" at 2024-08-10 09:58:29 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 223 / 26 / 250:  83%|█████████████████▌   | 250/300 [10:15<02:03,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723255236670.ta.chkpt" at 2024-08-10 10:00:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 268 / 31 / 300: 100%|█████████████████████| 300/300 [12:14<00:00,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1723255355416.ta.chkpt" at 2024-08-10 10:02:35 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 268 / 31 / 300: 100%|█████████████████████| 300/300 [12:14<00:00,  2.45s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 268    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.33% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 3.23%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.59  |
+-------------------------------+--------+


In [166]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 34 / 15 / 50:  17%|████                    | 50/300 [01:34<07:54,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723255451337.ta.chkpt" at 2024-08-10 10:04:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 67 / 30 / 100:  33%|███████▎              | 100/300 [03:01<06:03,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1723255537985.ta.chkpt" at 2024-08-10 10:05:37 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 67 / 31 / 101:  34%|███████▍              | 101/300 [03:01<05:58,  1.80s/it]

[Succeeded / Failed / Skipped / Total] 5 / 103 / 42 / 150:  50%|██████████▌          | 150/300 [04:38<04:38,  1.86s/it]textattack: Saving checkpoint under "checkpoints\1723255634979.ta.chkpt" at 2024-08-10 10:07:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 135 / 60 / 200:  67%|██████████████       | 200/300 [06:13<03:06,  1.87s/it]textattack: Saving checkpoint under "checkpoints\1723255730128.ta.chkpt" at 2024-08-10 10:08:50 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 173 / 71 / 250:  83%|█████████████████▌   | 250/300 [07:56<01:35,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723255832437.ta.chkpt" at 2024-08-10 10:10:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 210 / 84 / 300: 100%|█████████████████████| 300/300 [09:34<00:00,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723255930511.ta.chkpt" at 2024-08-10 10:12:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 210 / 84 / 300: 100%|█████████████████████| 300/300 [09:34<00:00,  1.91s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 6     |
| Number of failed attacks:     | 210   |
| Number of skipped attacks:    | 84    |
| Original accuracy:            | 72.0% |
| Accuracy under attack:        | 70.0% |
| Attack success rate:          | 2.78% |
| Average perturbed word %:     | 3.6%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 29.6  |
+-------------------------------+-------+


In [167]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:51<09:15,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723256041860.ta.chkpt" at 2024-08-10 10:14:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 76 / 23 / 100:  33%|███████▎              | 100/300 [03:21<06:43,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723256132574.ta.chkpt" at 2024-08-10 10:15:32 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 76 / 24 / 101:  34%|███████▍              | 101/300 [03:21<06:37,  2.00s/it]

[Succeeded / Failed / Skipped / Total] 1 / 117 / 32 / 150:  50%|██████████▌          | 150/300 [05:05<05:05,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723256236083.ta.chkpt" at 2024-08-10 10:17:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 152 / 47 / 200:  67%|██████████████       | 200/300 [06:48<03:24,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723256339754.ta.chkpt" at 2024-08-10 10:18:59 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 193 / 54 / 250:  83%|█████████████████▌   | 250/300 [08:43<01:44,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723256454203.ta.chkpt" at 2024-08-10 10:20:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 236 / 61 / 300: 100%|█████████████████████| 300/300 [10:40<00:00,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723256571075.ta.chkpt" at 2024-08-10 10:22:51 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 236 / 61 / 300: 100%|█████████████████████| 300/300 [10:40<00:00,  2.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 236    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 78.67% |
| Attack success rate:          | 1.26%  |
| Average perturbed word %:     | 4.2%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.65  |
+-------------------------------+--------+


In [168]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 42 / 8 / 50:  17%|████▏                    | 50/300 [01:51<09:15,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723256682402.ta.chkpt" at 2024-08-10 10:24:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 79 / 19 / 100:  33%|███████▎              | 100/300 [03:31<07:03,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723256783022.ta.chkpt" at 2024-08-10 10:26:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 120 / 27 / 150:  50%|██████████▌          | 150/300 [05:19<05:19,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723256891251.ta.chkpt" at 2024-08-10 10:28:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 157 / 39 / 200:  67%|██████████████       | 200/300 [07:06<03:33,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723256997904.ta.chkpt" at 2024-08-10 10:29:57 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 198 / 45 / 250:  83%|█████████████████▌   | 250/300 [09:03<01:48,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723257114931.ta.chkpt" at 2024-08-10 10:31:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 240 / 53 / 300: 100%|█████████████████████| 300/300 [11:02<00:00,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723257234146.ta.chkpt" at 2024-08-10 10:33:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 240 / 53 / 300: 100%|█████████████████████| 300/300 [11:02<00:00,  2.21s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 240    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 2.83%  |
| Average perturbed word %:     | 3.24%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.59  |
+-------------------------------+--------+


In [169]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 38 / 11 / 50:  17%|███▋                  | 50/300 [26:27<2:12:15, 31.74s/it]textattack: Saving checkpoint under "checkpoints\1723258821421.ta.chkpt" at 2024-08-10 11:00:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 76 / 20 / 100:  33%|███████▎              | 100/300 [28:30<57:01, 17.11s/it]textattack: Saving checkpoint under "checkpoints\1723258945027.ta.chkpt" at 2024-08-10 11:02:25 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 113 / 32 / 150:  50%|██████████▌          | 150/300 [30:30<30:30, 12.20s/it]textattack: Saving checkpoint under "checkpoints\1723259064728.ta.chkpt" at 2024-08-10 11:04:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 148 / 45 / 200:  67%|██████████████       | 200/300 [32:40<16:20,  9.80s/it]textattack: Saving checkpoint under "checkpoints\1723259194947.ta.chkpt" at 2024-08-10 11:06:34 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 189 / 52 / 250:  83%|█████████████████▌   | 250/300 [34:52<06:58,  8.37s/it]textattack: Saving checkpoint under "checkpoints\1723259326616.ta.chkpt" at 2024-08-10 11:08:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 233 / 58 / 300: 100%|█████████████████████| 300/300 [37:18<00:00,  7.46s/it]textattack: Saving checkpoint under "checkpoints\1723259472294.ta.chkpt" at 2024-08-10 11:11:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 233 / 58 / 300: 100%|█████████████████████| 300/300 [37:18<00:00,  7.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 3.72%  |
| Average perturbed word %:     | 2.69%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.64  |
+-------------------------------+--------+


In [170]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [02:13<11:09,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1723259607178.ta.chkpt" at 2024-08-10 11:13:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 79 / 19 / 100:  33%|███████▎              | 100/300 [04:12<08:24,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723259725549.ta.chkpt" at 2024-08-10 11:15:25 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 119 / 28 / 150:  50%|██████████▌          | 150/300 [06:18<06:18,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723259851405.ta.chkpt" at 2024-08-10 11:17:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 153 / 43 / 200:  67%|██████████████       | 200/300 [08:18<04:09,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723259971871.ta.chkpt" at 2024-08-10 11:19:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 194 / 51 / 250:  83%|█████████████████▌   | 250/300 [10:29<02:05,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1723260102699.ta.chkpt" at 2024-08-10 11:21:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 238 / 56 / 300: 100%|█████████████████████| 300/300 [12:51<00:00,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1723260245165.ta.chkpt" at 2024-08-10 11:24:05 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 238 / 56 / 300: 100%|█████████████████████| 300/300 [12:51<00:00,  2.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 238    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 79.33% |
| Attack success rate:          | 2.46%  |
| Average perturbed word %:     | 3.29%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.57  |
+-------------------------------+--------+


In [171]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [02:05<10:26,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723260370563.ta.chkpt" at 2024-08-10 11:26:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 78 / 20 / 100:  33%|███████▎              | 100/300 [04:00<08:01,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723260486127.ta.chkpt" at 2024-08-10 11:28:06 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 113 / 33 / 150:  50%|██████████▌          | 150/300 [05:55<05:55,  2.37s/it]textattack: Saving checkpoint under "checkpoints\1723260600921.ta.chkpt" at 2024-08-10 11:30:00 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 149 / 47 / 200:  67%|██████████████       | 200/300 [08:02<04:01,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723260727810.ta.chkpt" at 2024-08-10 11:32:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 190 / 54 / 250:  83%|█████████████████▌   | 250/300 [10:16<02:03,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1723260861423.ta.chkpt" at 2024-08-10 11:34:21 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 233 / 60 / 300: 100%|█████████████████████| 300/300 [12:40<00:00,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1723261005967.ta.chkpt" at 2024-08-10 11:36:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 233 / 60 / 300: 100%|█████████████████████| 300/300 [12:40<00:00,  2.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 2.92%  |
| Average perturbed word %:     | 3.12%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.47  |
+-------------------------------+--------+


#### CLIP

In [172]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:32<07:40,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723261098180.ta.chkpt" at 2024-08-10 11:38:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:50<05:41,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723261176674.ta.chkpt" at 2024-08-10 11:39:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:14<04:14,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723261260784.ta.chkpt" at 2024-08-10 11:41:00 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:33<02:46,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723261339684.ta.chkpt" at 2024-08-10 11:42:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [07:00<01:24,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1723261426458.ta.chkpt" at 2024-08-10 11:43:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:24<00:00,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1723261510637.ta.chkpt" at 2024-08-10 11:45:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:24<00:00,  1.68s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.65  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [173]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:19<06:37,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723261590267.ta.chkpt" at 2024-08-10 11:46:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:37<05:15,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723261668571.ta.chkpt" at 2024-08-10 11:47:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:00<04:00,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723261750892.ta.chkpt" at 2024-08-10 11:49:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:17<02:38,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723261828086.ta.chkpt" at 2024-08-10 11:50:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [06:46<01:21,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723261916815.ta.chkpt" at 2024-08-10 11:51:56 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:11<00:00,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723262002212.ta.chkpt" at 2024-08-10 11:53:22 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:11<00:00,  1.64s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.65  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [174]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:18<06:34,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723262081343.ta.chkpt" at 2024-08-10 11:54:41 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:41<05:22,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723262163726.ta.chkpt" at 2024-08-10 11:56:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:06<04:06,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1723262249296.ta.chkpt" at 2024-08-10 11:57:29 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:22<02:41,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723262325162.ta.chkpt" at 2024-08-10 11:58:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [06:50<01:22,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723262413295.ta.chkpt" at 2024-08-10 12:00:13 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:16<00:00,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1723262498819.ta.chkpt" at 2024-08-10 12:01:38 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:16<00:00,  1.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.65  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [175]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:18<06:33,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1723262577673.ta.chkpt" at 2024-08-10 12:02:57 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:30<05:00,  1.50s/it]textattack: Saving checkpoint under "checkpoints\1723262649284.ta.chkpt" at 2024-08-10 12:04:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [03:47<03:47,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1723262726416.ta.chkpt" at 2024-08-10 12:05:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 162 / 37 / 200:  67%|██████████████       | 200/300 [04:46<02:23,  1.43s/it]textattack: Saving checkpoint under "checkpoints\1723262785765.ta.chkpt" at 2024-08-10 12:06:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 204 / 45 / 250:  83%|█████████████████▌   | 250/300 [06:07<01:13,  1.47s/it]textattack: Saving checkpoint under "checkpoints\1723262866789.ta.chkpt" at 2024-08-10 12:07:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:28<00:00,  1.49s/it]textattack: Saving checkpoint under "checkpoints\1723262947218.ta.chkpt" at 2024-08-10 12:09:07 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:28<00:00,  1.49s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 248    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 82.67% |
| Attack success rate:          | 0.4%   |
| Average perturbed word %:     | 1.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.4   |
+-------------------------------+--------+


In [176]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:26<07:12,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723263033746.ta.chkpt" at 2024-08-10 12:10:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [02:46<05:32,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1723263113721.ta.chkpt" at 2024-08-10 12:11:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  50%|██████████▌          | 150/300 [04:05<04:05,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723263193025.ta.chkpt" at 2024-08-10 12:13:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 173 / 25 / 200:  67%|██████████████       | 200/300 [05:08<02:34,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1723263255546.ta.chkpt" at 2024-08-10 12:14:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 217 / 31 / 250:  83%|█████████████████▌   | 250/300 [06:34<01:18,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723263342211.ta.chkpt" at 2024-08-10 12:15:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 262 / 36 / 300: 100%|█████████████████████| 300/300 [08:03<00:00,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723263430543.ta.chkpt" at 2024-08-10 12:17:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 262 / 36 / 300: 100%|█████████████████████| 300/300 [08:03<00:00,  1.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 0.76%  |
| Average perturbed word %:     | 2.58%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.33  |
+-------------------------------+--------+


In [177]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:31<07:39,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1723263522460.ta.chkpt" at 2024-08-10 12:18:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [02:52<05:45,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1723263603187.ta.chkpt" at 2024-08-10 12:20:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 139 / 11 / 150:  50%|██████████▌          | 150/300 [04:16<04:16,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1723263686664.ta.chkpt" at 2024-08-10 12:21:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [05:27<02:43,  1.64s/it]textattack: Saving checkpoint under "checkpoints\1723263758274.ta.chkpt" at 2024-08-10 12:22:38 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 224 / 26 / 250:  83%|█████████████████▌   | 250/300 [06:52<01:22,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1723263842860.ta.chkpt" at 2024-08-10 12:24:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [08:17<00:00,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1723263928570.ta.chkpt" at 2024-08-10 12:25:28 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [08:18<00:00,  1.66s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.37  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [178]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [25:05<2:05:26, 30.11s/it]textattack: Saving checkpoint under "checkpoints\1723265434095.ta.chkpt" at 2024-08-10 12:50:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [26:44<53:28, 16.04s/it]textattack: Saving checkpoint under "checkpoints\1723265532808.ta.chkpt" at 2024-08-10 12:52:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [28:23<28:23, 11.36s/it]textattack: Saving checkpoint under "checkpoints\1723265632443.ta.chkpt" at 2024-08-10 12:53:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [29:57<14:58,  8.99s/it]textattack: Saving checkpoint under "checkpoints\1723265726060.ta.chkpt" at 2024-08-10 12:55:26 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 233 / 17 / 250:  83%|█████████████████▌   | 250/300 [31:45<06:21,  7.62s/it]textattack: Saving checkpoint under "checkpoints\1723265833985.ta.chkpt" at 2024-08-10 12:57:13 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [33:24<00:00,  6.68s/it]textattack: Saving checkpoint under "checkpoints\1723265933688.ta.chkpt" at 2024-08-10 12:58:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [33:24<00:00,  6.68s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.36  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [179]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:41<08:27,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1723266035324.ta.chkpt" at 2024-08-10 13:00:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:23<06:47,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723266137255.ta.chkpt" at 2024-08-10 13:02:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:01<05:01,  2.01s/it]textattack: Saving checkpoint under "checkpoints\1723266234993.ta.chkpt" at 2024-08-10 13:03:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [06:33<03:16,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1723266327423.ta.chkpt" at 2024-08-10 13:05:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [08:20<01:40,  2.00s/it]textattack: Saving checkpoint under "checkpoints\1723266433785.ta.chkpt" at 2024-08-10 13:07:13 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [09:59<00:00,  2.00s/it]textattack: Saving checkpoint under "checkpoints\1723266532795.ta.chkpt" at 2024-08-10 13:08:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [09:59<00:00,  2.00s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.51  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [180]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:40<08:22,  2.01s/it]textattack: Saving checkpoint under "checkpoints\1723266633385.ta.chkpt" at 2024-08-10 13:10:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 95 / 5 / 100:  33%|███████▋               | 100/300 [03:24<06:48,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723266737385.ta.chkpt" at 2024-08-10 13:12:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 141 / 9 / 150:  50%|███████████           | 150/300 [05:03<05:03,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723266836067.ta.chkpt" at 2024-08-10 13:13:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [06:20<03:10,  1.90s/it]textattack: Saving checkpoint under "checkpoints\1723266913552.ta.chkpt" at 2024-08-10 13:15:13 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 222 / 28 / 250:  83%|█████████████████▌   | 250/300 [08:02<01:36,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723267015604.ta.chkpt" at 2024-08-10 13:16:55 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [09:40<00:00,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723267113228.ta.chkpt" at 2024-08-10 13:18:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [09:40<00:00,  1.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.2   |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [181]:
et8 = datetime.datetime.now()
et8 - et7

datetime.timedelta(seconds=13876, microseconds=723476)

### MinSim 0.9, threshold 0.5

In [182]:
suffix = 'sim09.csv'
thre = 1-0.5/np.pi
minsim = 0.9

#### BERT

In [183]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr1sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:48<09:01,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723267222069.ta.chkpt" at 2024-08-10 13:20:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 90 / 10 / 100:  33%|███████▎              | 100/300 [03:39<07:19,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723267333346.ta.chkpt" at 2024-08-10 13:22:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 135 / 15 / 150:  50%|██████████▌          | 150/300 [05:34<05:34,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723267448311.ta.chkpt" at 2024-08-10 13:24:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [07:36<03:48,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723267570285.ta.chkpt" at 2024-08-10 13:26:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 225 / 25 / 250:  83%|█████████████████▌   | 250/300 [09:38<01:55,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1723267692611.ta.chkpt" at 2024-08-10 13:28:12 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 270 / 30 / 300: 100%|█████████████████████| 300/300 [11:40<00:00,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1723267814551.ta.chkpt" at 2024-08-10 13:30:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 270 / 30 / 300: 100%|█████████████████████| 300/300 [11:40<00:00,  2.34s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 270   |
| Number of skipped attacks:    | 30    |
| Original accuracy:            | 90.0% |
| Accuracy under attack:        | 90.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 39.3  |
| Avg num queries:              | 29.34 |
+-------------------------------+-------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [184]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr01sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:53<09:28,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723267928445.ta.chkpt" at 2024-08-10 13:32:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [03:39<07:18,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723268033750.ta.chkpt" at 2024-08-10 13:33:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 134 / 16 / 150:  50%|██████████▌          | 150/300 [05:30<05:30,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723268145419.ta.chkpt" at 2024-08-10 13:35:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 177 / 23 / 200:  67%|██████████████       | 200/300 [07:29<03:44,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723268263994.ta.chkpt" at 2024-08-10 13:37:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 223 / 27 / 250:  83%|█████████████████▌   | 250/300 [09:26<01:53,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723268380864.ta.chkpt" at 2024-08-10 13:39:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [11:25<00:00,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723268500439.ta.chkpt" at 2024-08-10 13:41:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [11:25<00:00,  2.29s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 268    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 89.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.31  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [185]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattacklr10sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [01:52<09:22,  2.25s/it]textattack: Saving checkpoint under "checkpoints\1723268613081.ta.chkpt" at 2024-08-10 13:43:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [03:39<07:18,  2.19s/it]textattack: Saving checkpoint under "checkpoints\1723268719934.ta.chkpt" at 2024-08-10 13:45:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 134 / 16 / 150:  50%|██████████▌          | 150/300 [05:30<05:30,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723268831410.ta.chkpt" at 2024-08-10 13:47:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 178 / 22 / 200:  67%|██████████████       | 200/300 [07:26<03:43,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1723268947547.ta.chkpt" at 2024-08-10 13:49:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 224 / 26 / 250:  83%|█████████████████▌   | 250/300 [09:24<01:52,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723269065495.ta.chkpt" at 2024-08-10 13:51:05 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [11:23<00:00,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1723269183676.ta.chkpt" at 2024-08-10 13:53:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [11:23<00:00,  2.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.32  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [186]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 35 / 15 / 50:  17%|████                    | 50/300 [01:29<07:26,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723269273119.ta.chkpt" at 2024-08-10 13:54:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 69 / 30 / 100:  33%|███████▎              | 100/300 [02:53<05:47,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1723269357643.ta.chkpt" at 2024-08-10 13:55:57 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 69 / 30 / 100:  34%|███████▍              | 101/300 [02:54<05:42,  1.72s/it]

[Succeeded / Failed / Skipped / Total] 2 / 106 / 42 / 150:  50%|██████████▌          | 150/300 [04:28<04:28,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1723269452090.ta.chkpt" at 2024-08-10 13:57:32 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 138 / 60 / 200:  67%|██████████████       | 200/300 [06:10<03:05,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1723269554132.ta.chkpt" at 2024-08-10 13:59:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 177 / 71 / 250:  83%|█████████████████▌   | 250/300 [07:57<01:35,  1.91s/it]textattack: Saving checkpoint under "checkpoints\1723269660997.ta.chkpt" at 2024-08-10 14:01:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 214 / 84 / 300: 100%|█████████████████████| 300/300 [09:37<00:00,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1723269761350.ta.chkpt" at 2024-08-10 14:02:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 214 / 84 / 300: 100%|█████████████████████| 300/300 [09:37<00:00,  1.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 214    |
| Number of skipped attacks:    | 84     |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 71.33% |
| Attack success rate:          | 0.93%  |
| Average perturbed word %:     | 2.74%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.33  |
+-------------------------------+--------+


In [187]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf14sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:01<10:07,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1723269883971.ta.chkpt" at 2024-08-10 14:04:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 77 / 23 / 100:  33%|███████▎              | 100/300 [03:29<06:59,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1723269972198.ta.chkpt" at 2024-08-10 14:06:12 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 77 / 24 / 101:  34%|███████▍              | 101/300 [03:29<06:53,  2.08s/it]

[Succeeded / Failed / Skipped / Total] 0 / 118 / 32 / 150:  50%|██████████▌          | 150/300 [05:12<05:12,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723270075281.ta.chkpt" at 2024-08-10 14:07:55 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 153 / 47 / 200:  67%|██████████████       | 200/300 [07:07<03:33,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1723270189736.ta.chkpt" at 2024-08-10 14:09:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 195 / 54 / 250:  83%|█████████████████▌   | 250/300 [09:02<01:48,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723270305239.ta.chkpt" at 2024-08-10 14:11:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 238 / 61 / 300: 100%|█████████████████████| 300/300 [10:58<00:00,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723270421374.ta.chkpt" at 2024-08-10 14:13:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 238 / 61 / 300: 100%|█████████████████████| 300/300 [10:58<00:00,  2.20s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 238    |
| Number of skipped attacks:    | 61     |
| Original accuracy:            | 79.67% |
| Accuracy under attack:        | 79.33% |
| Attack success rate:          | 0.42%  |
| Average perturbed word %:     | 3.12%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.4   |
+-------------------------------+--------+


In [188]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackrf30sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 42 / 8 / 50:  17%|████▏                    | 50/300 [02:02<10:10,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1723270545428.ta.chkpt" at 2024-08-10 14:15:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 81 / 19 / 100:  33%|███████▎              | 100/300 [03:36<07:13,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723270640127.ta.chkpt" at 2024-08-10 14:17:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 122 / 27 / 150:  50%|██████████▌          | 150/300 [05:20<05:20,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1723270743809.ta.chkpt" at 2024-08-10 14:19:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 160 / 39 / 200:  67%|██████████████       | 200/300 [07:05<03:32,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1723270848814.ta.chkpt" at 2024-08-10 14:20:48 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 203 / 45 / 250:  83%|█████████████████▌   | 250/300 [08:58<01:47,  2.15s/it]textattack: Saving checkpoint under "checkpoints\1723270961765.ta.chkpt" at 2024-08-10 14:22:41 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 245 / 53 / 300: 100%|█████████████████████| 300/300 [10:50<00:00,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1723271073527.ta.chkpt" at 2024-08-10 14:24:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 245 / 53 / 300: 100%|█████████████████████| 300/300 [10:50<00:00,  2.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 245    |
| Number of skipped attacks:    | 53     |
| Original accuracy:            | 82.33% |
| Accuracy under attack:        | 81.67% |
| Attack success rate:          | 0.81%  |
| Average perturbed word %:     | 2.91%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.32  |
+-------------------------------+--------+


In [189]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 38 / 11 / 50:  17%|███▋                  | 50/300 [26:46<2:13:51, 32.13s/it]textattack: Saving checkpoint under "checkpoints\1723272680041.ta.chkpt" at 2024-08-10 14:51:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 78 / 20 / 100:  33%|███████▎              | 100/300 [28:45<57:31, 17.26s/it]textattack: Saving checkpoint under "checkpoints\1723272799618.ta.chkpt" at 2024-08-10 14:53:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 115 / 32 / 150:  50%|██████████▌          | 150/300 [30:45<30:45, 12.31s/it]textattack: Saving checkpoint under "checkpoints\1723272919599.ta.chkpt" at 2024-08-10 14:55:19 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 152 / 45 / 200:  67%|██████████████       | 200/300 [33:04<16:32,  9.92s/it]textattack: Saving checkpoint under "checkpoints\1723273058120.ta.chkpt" at 2024-08-10 14:57:38 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 195 / 52 / 250:  83%|█████████████████▌   | 250/300 [35:20<07:04,  8.48s/it]textattack: Saving checkpoint under "checkpoints\1723273194610.ta.chkpt" at 2024-08-10 14:59:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 239 / 58 / 300: 100%|█████████████████████| 300/300 [37:46<00:00,  7.55s/it]textattack: Saving checkpoint under "checkpoints\1723273339747.ta.chkpt" at 2024-08-10 15:02:19 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 239 / 58 / 300: 100%|█████████████████████| 300/300 [37:46<00:00,  7.55s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 239    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 79.67% |
| Attack success rate:          | 1.24%  |
| Average perturbed word %:     | 2.71%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.38  |
+-------------------------------+--------+


In [190]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn35sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:10<10:52,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1723273470993.ta.chkpt" at 2024-08-10 15:04:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 81 / 19 / 100:  33%|███████▎              | 100/300 [04:10<08:20,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723273590956.ta.chkpt" at 2024-08-10 15:06:30 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 121 / 28 / 150:  50%|██████████▌          | 150/300 [06:14<06:14,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723273714727.ta.chkpt" at 2024-08-10 15:08:34 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 155 / 43 / 200:  67%|██████████████       | 200/300 [08:17<04:08,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1723273838193.ta.chkpt" at 2024-08-10 15:10:38 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 197 / 51 / 250:  83%|█████████████████▌   | 250/300 [10:24<02:04,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1723273964725.ta.chkpt" at 2024-08-10 15:12:44 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 242 / 56 / 300: 100%|█████████████████████| 300/300 [12:45<00:00,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1723274106465.ta.chkpt" at 2024-08-10 15:15:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 242 / 56 / 300: 100%|█████████████████████| 300/300 [12:45<00:00,  2.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 242    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 80.67% |
| Attack success rate:          | 0.82%  |
| Average perturbed word %:     | 4.76%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.29  |
+-------------------------------+--------+


In [191]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackknn175sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [01:54<09:33,  2.29s/it]textattack: Saving checkpoint under "checkpoints\1723274221191.ta.chkpt" at 2024-08-10 15:17:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 80 / 20 / 100:  33%|███████▎              | 100/300 [03:46<07:32,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723274332972.ta.chkpt" at 2024-08-10 15:18:52 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 116 / 33 / 150:  50%|██████████▌          | 150/300 [05:38<05:38,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1723274444827.ta.chkpt" at 2024-08-10 15:20:44 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 152 / 47 / 200:  67%|██████████████       | 200/300 [07:39<03:49,  2.30s/it]textattack: Saving checkpoint under "checkpoints\1723274566104.ta.chkpt" at 2024-08-10 15:22:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 194 / 54 / 250:  83%|█████████████████▌   | 250/300 [09:46<01:57,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1723274692866.ta.chkpt" at 2024-08-10 15:24:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 238 / 60 / 300: 100%|█████████████████████| 300/300 [12:03<00:00,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1723274829819.ta.chkpt" at 2024-08-10 15:27:09 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 238 / 60 / 300: 100%|█████████████████████| 300/300 [12:03<00:00,  2.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 238    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 79.33% |
| Attack success rate:          | 0.83%  |
| Average perturbed word %:     | 2.26%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.21  |
+-------------------------------+--------+


#### CLIP

In [192]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr1sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:23<06:57,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723274913387.ta.chkpt" at 2024-08-10 15:28:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:38<05:17,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723274988519.ta.chkpt" at 2024-08-10 15:29:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:00<04:00,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723275070760.ta.chkpt" at 2024-08-10 15:31:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:14<02:37,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1723275144295.ta.chkpt" at 2024-08-10 15:32:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [06:36<01:19,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723275226646.ta.chkpt" at 2024-08-10 15:33:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [07:58<00:00,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723275308874.ta.chkpt" at 2024-08-10 15:35:08 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [07:59<00:00,  1.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.38  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [193]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr01sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:23<06:56,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1723275392273.ta.chkpt" at 2024-08-10 15:36:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:40<05:20,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723275469263.ta.chkpt" at 2024-08-10 15:37:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [04:01<04:01,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723275550638.ta.chkpt" at 2024-08-10 15:39:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:16<02:38,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723275625349.ta.chkpt" at 2024-08-10 15:40:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [06:41<01:20,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1723275710529.ta.chkpt" at 2024-08-10 15:41:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:06<00:00,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723275795288.ta.chkpt" at 2024-08-10 15:43:15 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [08:06<00:00,  1.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.38  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [194]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattacklr10sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:20<06:40,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723275875479.ta.chkpt" at 2024-08-10 15:44:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [02:38<05:17,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723275954126.ta.chkpt" at 2024-08-10 15:45:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [03:58<03:58,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1723276034260.ta.chkpt" at 2024-08-10 15:47:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 180 / 20 / 200:  67%|██████████████       | 200/300 [05:11<02:35,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1723276106954.ta.chkpt" at 2024-08-10 15:48:26 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 226 / 24 / 250:  83%|█████████████████▌   | 250/300 [06:34<01:18,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723276190298.ta.chkpt" at 2024-08-10 15:49:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [07:59<00:00,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723276275050.ta.chkpt" at 2024-08-10 15:51:15 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 271 / 29 / 300: 100%|█████████████████████| 300/300 [07:59<00:00,  1.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 271    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.38  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [195]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [01:17<06:28,  1.55s/it]textattack: Saving checkpoint under "checkpoints\1723276352839.ta.chkpt" at 2024-08-10 15:52:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 85 / 15 / 100:  33%|███████▎              | 100/300 [02:26<04:53,  1.47s/it]textattack: Saving checkpoint under "checkpoints\1723276421830.ta.chkpt" at 2024-08-10 15:53:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 127 / 23 / 150:  50%|██████████▌          | 150/300 [03:41<03:41,  1.48s/it]textattack: Saving checkpoint under "checkpoints\1723276496666.ta.chkpt" at 2024-08-10 15:54:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 162 / 37 / 200:  67%|██████████████       | 200/300 [04:40<02:20,  1.40s/it]textattack: Saving checkpoint under "checkpoints\1723276555545.ta.chkpt" at 2024-08-10 15:55:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 204 / 45 / 250:  83%|█████████████████▌   | 250/300 [05:58<01:11,  1.44s/it]textattack: Saving checkpoint under "checkpoints\1723276634019.ta.chkpt" at 2024-08-10 15:57:14 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:15<00:00,  1.45s/it]textattack: Saving checkpoint under "checkpoints\1723276711127.ta.chkpt" at 2024-08-10 15:58:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 248 / 51 / 300: 100%|█████████████████████| 300/300 [07:15<00:00,  1.45s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 248    |
| Number of skipped attacks:    | 51     |
| Original accuracy:            | 83.0%  |
| Accuracy under attack:        | 82.67% |
| Attack success rate:          | 0.4%   |
| Average perturbed word %:     | 1.89%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.13  |
+-------------------------------+--------+


In [196]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf14sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [01:21<06:48,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1723276792956.ta.chkpt" at 2024-08-10 15:59:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [02:38<05:16,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1723276869571.ta.chkpt" at 2024-08-10 16:01:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 136 / 14 / 150:  50%|██████████▌          | 150/300 [03:54<03:54,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1723276945784.ta.chkpt" at 2024-08-10 16:02:25 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 174 / 25 / 200:  67%|██████████████       | 200/300 [04:55<02:27,  1.48s/it]textattack: Saving checkpoint under "checkpoints\1723277007164.ta.chkpt" at 2024-08-10 16:03:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 218 / 31 / 250:  83%|█████████████████▌   | 250/300 [06:17<01:15,  1.51s/it]textattack: Saving checkpoint under "checkpoints\1723277088891.ta.chkpt" at 2024-08-10 16:04:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 263 / 36 / 300: 100%|█████████████████████| 300/300 [07:42<00:00,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1723277173324.ta.chkpt" at 2024-08-10 16:06:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 263 / 36 / 300: 100%|█████████████████████| 300/300 [07:42<00:00,  1.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 263    |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 87.67% |
| Attack success rate:          | 0.38%  |
| Average perturbed word %:     | 2.13%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.07  |
+-------------------------------+--------+


In [197]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackrf30sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 47 / 3 / 50:  17%|████▏                    | 50/300 [01:24<07:04,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1723277258453.ta.chkpt" at 2024-08-10 16:07:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 92 / 8 / 100:  33%|███████▋               | 100/300 [02:42<05:24,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1723277335867.ta.chkpt" at 2024-08-10 16:08:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 139 / 11 / 150:  50%|██████████▌          | 150/300 [04:00<04:00,  1.60s/it]textattack: Saving checkpoint under "checkpoints\1723277413890.ta.chkpt" at 2024-08-10 16:10:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [05:09<02:34,  1.55s/it]textattack: Saving checkpoint under "checkpoints\1723277482692.ta.chkpt" at 2024-08-10 16:11:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 224 / 26 / 250:  83%|█████████████████▌   | 250/300 [06:29<01:17,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1723277562621.ta.chkpt" at 2024-08-10 16:12:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [07:49<00:00,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1723277643362.ta.chkpt" at 2024-08-10 16:14:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 269 / 31 / 300: 100%|█████████████████████| 300/300 [07:49<00:00,  1.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.1   |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [198]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [25:15<2:06:15, 30.30s/it]textattack: Saving checkpoint under "checkpoints\1723279158671.ta.chkpt" at 2024-08-10 16:39:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  33%|███████▋               | 100/300 [27:13<54:27, 16.34s/it]textattack: Saving checkpoint under "checkpoints\1723279277265.ta.chkpt" at 2024-08-10 16:41:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 143 / 7 / 150:  50%|███████████           | 150/300 [29:04<29:04, 11.63s/it]textattack: Saving checkpoint under "checkpoints\1723279387620.ta.chkpt" at 2024-08-10 16:43:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 186 / 14 / 200:  67%|██████████████       | 200/300 [30:38<15:19,  9.19s/it]textattack: Saving checkpoint under "checkpoints\1723279481559.ta.chkpt" at 2024-08-10 16:44:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 233 / 17 / 250:  83%|█████████████████▌   | 250/300 [32:36<06:31,  7.83s/it]textattack: Saving checkpoint under "checkpoints\1723279600184.ta.chkpt" at 2024-08-10 16:46:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [34:17<00:00,  6.86s/it]textattack: Saving checkpoint under "checkpoints\1723279701320.ta.chkpt" at 2024-08-10 16:48:21 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [34:17<00:00,  6.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.08  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [199]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn35sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [01:41<08:25,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723279802515.ta.chkpt" at 2024-08-10 16:50:02 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [03:22<06:45,  2.03s/it]textattack: Saving checkpoint under "checkpoints\1723279904193.ta.chkpt" at 2024-08-10 16:51:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [05:02<05:02,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1723280003874.ta.chkpt" at 2024-08-10 16:53:23 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [06:47<03:23,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1723280108868.ta.chkpt" at 2024-08-10 16:55:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [08:42<01:44,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723280224135.ta.chkpt" at 2024-08-10 16:57:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:34<00:00,  2.12s/it]textattack: Saving checkpoint under "checkpoints\1723280336252.ta.chkpt" at 2024-08-10 16:58:56 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 278 / 22 / 300: 100%|█████████████████████| 300/300 [10:34<00:00,  2.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 278    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.25  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [200]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/clipagattackknn175sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [01:50<09:11,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1723280446660.ta.chkpt" at 2024-08-10 17:00:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 95 / 5 / 100:  33%|███████▋               | 100/300 [03:42<07:24,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1723280558834.ta.chkpt" at 2024-08-10 17:02:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 141 / 9 / 150:  50%|███████████           | 150/300 [05:30<05:30,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1723280666771.ta.chkpt" at 2024-08-10 17:04:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [06:58<03:29,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723280755151.ta.chkpt" at 2024-08-10 17:05:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 222 / 28 / 250:  83%|█████████████████▌   | 250/300 [08:41<01:44,  2.09s/it]textattack: Saving checkpoint under "checkpoints\1723280857683.ta.chkpt" at 2024-08-10 17:07:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [10:17<00:00,  2.06s/it]textattack: Saving checkpoint under "checkpoints\1723280953450.ta.chkpt" at 2024-08-10 17:09:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [10:17<00:00,  2.06s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 28.93  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [201]:
et9 = datetime.datetime.now()
et9 - et8

datetime.timedelta(seconds=13840, microseconds=253031)

### Fine-tuned

In [202]:
# bert-based-uncased
bert_ft_path = 'textattack/bert-base-uncased-ag-news'

model_bert_ft = transformers.AutoModelForSequenceClassification.from_pretrained(bert_ft_path)
tokenizer_bert_ft = transformers.AutoTokenizer.from_pretrained(bert_ft_path)
wrapper_bert_ft = HuggingFaceModelWrapper(model_bert_ft, tokenizer_bert_ft)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [203]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0809.csv', target_ds_ag, 0.5, 1-0.6/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|███▊                   | 50/300 [31:23<2:36:59, 37.68s/it]textattack: Saving checkpoint under "checkpoints\1723284973813.ta.chkpt" at 2024-08-10 18:16:13 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  33%|██████▎            | 100/300 [1:03:14<2:06:28, 37.94s/it]textattack: Saving checkpoint under "checkpoints\1723286884227.ta.chkpt" at 2024-08-10 18:48:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 139 / 8 / 150:  50%|█████████         | 150/300 [1:33:06<1:33:06, 37.24s/it]textattack: Saving checkpoint under "checkpoints\1723288676110.ta.chkpt" at 2024-08-10 19:17:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200:  67%|███████████▎     | 200/300 [2:02:13<1:01:06, 36.67s/it]textattack: Saving checkpoint under "checkpoints\1723290423969.ta.chkpt" at 2024-08-10 19:47:03 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 230 / 15 / 250:  83%|███████████████▊   | 250/300 [2:31:55<30:23, 36.46s/it]textattack: Saving checkpoint under "checkpoints\1723292205753.ta.chkpt" at 2024-08-10 20:16:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 275 / 19 / 300: 100%|███████████████████| 300/300 [3:00:58<00:00, 36.20s/it]textattack: Saving checkpoint under "checkpoints\1723293948561.ta.chkpt" at 2024-08-10 20:45:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 275 / 19 / 300: 100%|███████████████████| 300/300 [3:00:58<00:00, 36.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 275    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 91.67% |
| Attack success rate:          | 2.14%  |
| Average perturbed word %:     | 3.18%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.75  |
+-------------------------------+--------+


In [204]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0841.csv', target_ds_ag, 0.5, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|███▊                   | 50/300 [28:19<2:21:35, 33.98s/it]textattack: Saving checkpoint under "checkpoints\1723295647719.ta.chkpt" at 2024-08-10 21:14:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  33%|███████              | 100/300 [57:32<1:55:05, 34.53s/it]textattack: Saving checkpoint under "checkpoints\1723297401623.ta.chkpt" at 2024-08-10 21:43:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 139 / 8 / 150:  50%|█████████         | 150/300 [1:27:52<1:27:52, 35.15s/it]textattack: Saving checkpoint under "checkpoints\1723299221478.ta.chkpt" at 2024-08-10 22:13:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200:  67%|████████████▋      | 200/300 [1:57:24<58:42, 35.22s/it]textattack: Saving checkpoint under "checkpoints\1723300993220.ta.chkpt" at 2024-08-10 22:43:13 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 230 / 15 / 250:  83%|███████████████▊   | 250/300 [2:27:31<29:30, 35.41s/it]textattack: Saving checkpoint under "checkpoints\1723302799989.ta.chkpt" at 2024-08-10 23:13:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 275 / 19 / 300: 100%|███████████████████| 300/300 [2:56:25<00:00, 35.28s/it]textattack: Saving checkpoint under "checkpoints\1723304533798.ta.chkpt" at 2024-08-10 23:42:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 275 / 19 / 300: 100%|███████████████████| 300/300 [2:56:25<00:00, 35.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 275    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 91.67% |
| Attack success rate:          | 2.14%  |
| Average perturbed word %:     | 3.18%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.73  |
+-------------------------------+--------+


In [205]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0873.csv', target_ds_ag, 0.5, 1-0.4/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|███▊                   | 50/300 [27:35<2:17:59, 33.12s/it]textattack: Saving checkpoint under "checkpoints\1723306189784.ta.chkpt" at 2024-08-11 00:09:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  33%|███████              | 100/300 [56:19<1:52:38, 33.79s/it]textattack: Saving checkpoint under "checkpoints\1723307912974.ta.chkpt" at 2024-08-11 00:38:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 139 / 8 / 150:  50%|█████████         | 150/300 [1:25:32<1:25:32, 34.21s/it]textattack: Saving checkpoint under "checkpoints\1723309666049.ta.chkpt" at 2024-08-11 01:07:46 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200:  67%|████████████▋      | 200/300 [1:54:22<57:11, 34.31s/it]textattack: Saving checkpoint under "checkpoints\1723311396447.ta.chkpt" at 2024-08-11 01:36:36 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 231 / 15 / 250:  83%|███████████████▊   | 250/300 [2:24:15<28:51, 34.62s/it]textattack: Saving checkpoint under "checkpoints\1723313189898.ta.chkpt" at 2024-08-11 02:06:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 276 / 19 / 300: 100%|███████████████████| 300/300 [2:53:03<00:00, 34.61s/it]textattack: Saving checkpoint under "checkpoints\1723314917874.ta.chkpt" at 2024-08-11 02:35:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 276 / 19 / 300: 100%|███████████████████| 300/300 [2:53:04<00:00, 34.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 1.78%  |
| Average perturbed word %:     | 3.22%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.51  |
+-------------------------------+--------+


In [206]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep09.csv', target_ds_ag, 0.5, 0.9)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|███▊                   | 50/300 [27:48<2:19:04, 33.38s/it]textattack: Saving checkpoint under "checkpoints\1723316586962.ta.chkpt" at 2024-08-11 03:03:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 94 / 6 / 100:  33%|███████              | 100/300 [55:46<1:51:33, 33.47s/it]textattack: Saving checkpoint under "checkpoints\1723318264790.ta.chkpt" at 2024-08-11 03:31:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 141 / 8 / 150:  50%|█████████         | 150/300 [1:24:52<1:24:52, 33.95s/it]textattack: Saving checkpoint under "checkpoints\1723320010068.ta.chkpt" at 2024-08-11 04:00:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 188 / 11 / 200:  67%|████████████▋      | 200/300 [1:54:33<57:16, 34.37s/it]textattack: Saving checkpoint under "checkpoints\1723321791644.ta.chkpt" at 2024-08-11 04:29:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 233 / 15 / 250:  83%|███████████████▊   | 250/300 [2:23:42<28:44, 34.49s/it]textattack: Saving checkpoint under "checkpoints\1723323540603.ta.chkpt" at 2024-08-11 04:59:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 279 / 19 / 300: 100%|███████████████████| 300/300 [2:51:52<00:00, 34.38s/it]textattack: Saving checkpoint under "checkpoints\1723325230661.ta.chkpt" at 2024-08-11 05:27:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 279 / 19 / 300: 100%|███████████████████| 300/300 [2:51:52<00:00, 34.38s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 279    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.0%  |
| Attack success rate:          | 0.71%  |
| Average perturbed word %:     | 5.98%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.86  |
+-------------------------------+--------+


In [207]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep095.csv', target_ds_ag, 0.5, 0.95)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|███▊                   | 50/300 [26:29<2:12:28, 31.79s/it]textattack: Saving checkpoint under "checkpoints\1723326820813.ta.chkpt" at 2024-08-11 05:53:40 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 94 / 6 / 100:  33%|███████              | 100/300 [53:21<1:46:42, 32.01s/it]textattack: Saving checkpoint under "checkpoints\1723328432553.ta.chkpt" at 2024-08-11 06:20:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 142 / 8 / 150:  50%|█████████         | 150/300 [1:20:54<1:20:54, 32.36s/it]textattack: Saving checkpoint under "checkpoints\1723330085810.ta.chkpt" at 2024-08-11 06:48:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 189 / 11 / 200:  67%|████████████▋      | 200/300 [1:48:11<54:05, 32.46s/it]textattack: Saving checkpoint under "checkpoints\1723331722188.ta.chkpt" at 2024-08-11 07:15:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 235 / 15 / 250:  83%|███████████████▊   | 250/300 [2:15:35<27:07, 32.54s/it]textattack: Saving checkpoint under "checkpoints\1723333366305.ta.chkpt" at 2024-08-11 07:42:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 281 / 19 / 300: 100%|███████████████████| 300/300 [2:42:50<00:00, 32.57s/it]textattack: Saving checkpoint under "checkpoints\1723335001694.ta.chkpt" at 2024-08-11 08:10:01 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 281 / 19 / 300: 100%|███████████████████| 300/300 [2:42:50<00:00, 32.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 281    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.28  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [208]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim06.csv', target_ds_ag, 0.6, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptepsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|███▊                   | 50/300 [26:59<2:14:55, 32.38s/it]textattack: Saving checkpoint under "checkpoints\1723336620939.ta.chkpt" at 2024-08-11 08:37:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  33%|███████              | 100/300 [54:30<1:49:00, 32.70s/it]textattack: Saving checkpoint under "checkpoints\1723338272025.ta.chkpt" at 2024-08-11 09:04:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 139 / 8 / 150:  50%|█████████         | 150/300 [1:22:52<1:22:52, 33.15s/it]textattack: Saving checkpoint under "checkpoints\1723339974354.ta.chkpt" at 2024-08-11 09:32:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200:  67%|████████████▋      | 200/300 [1:51:07<55:33, 33.34s/it]textattack: Saving checkpoint under "checkpoints\1723341669590.ta.chkpt" at 2024-08-11 10:01:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 231 / 15 / 250:  83%|███████████████▊   | 250/300 [2:19:50<27:58, 33.56s/it]textattack: Saving checkpoint under "checkpoints\1723343392703.ta.chkpt" at 2024-08-11 10:29:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 276 / 19 / 300: 100%|███████████████████| 300/300 [2:47:45<00:00, 33.55s/it]textattack: Saving checkpoint under "checkpoints\1723345067801.ta.chkpt" at 2024-08-11 10:57:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 276 / 19 / 300: 100%|███████████████████| 300/300 [2:47:45<00:00, 33.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 276    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 92.0%  |
| Attack success rate:          | 1.78%  |
| Average perturbed word %:     | 3.22%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.74  |
+-------------------------------+--------+


In [209]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim07.csv', target_ds_ag, 0.7, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptepsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|███▊                   | 50/300 [26:37<2:13:07, 31.95s/it]textattack: Saving checkpoint under "checkpoints\1723346665402.ta.chkpt" at 2024-08-11 11:24:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  33%|███████              | 100/300 [53:32<1:47:04, 32.12s/it]textattack: Saving checkpoint under "checkpoints\1723348280037.ta.chkpt" at 2024-08-11 11:51:20 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 141 / 8 / 150:  50%|█████████         | 150/300 [1:21:33<1:21:33, 32.63s/it]textattack: Saving checkpoint under "checkpoints\1723349961783.ta.chkpt" at 2024-08-11 12:19:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 188 / 11 / 200:  67%|████████████▋      | 200/300 [1:49:00<54:30, 32.70s/it]textattack: Saving checkpoint under "checkpoints\1723351608038.ta.chkpt" at 2024-08-11 12:46:48 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 233 / 15 / 250:  83%|███████████████▊   | 250/300 [2:16:11<27:14, 32.69s/it]textattack: Saving checkpoint under "checkpoints\1723353239444.ta.chkpt" at 2024-08-11 13:13:59 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 279 / 19 / 300: 100%|███████████████████| 300/300 [2:43:14<00:00, 32.65s/it]textattack: Saving checkpoint under "checkpoints\1723354862540.ta.chkpt" at 2024-08-11 13:41:02 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 279 / 19 / 300: 100%|███████████████████| 300/300 [2:43:14<00:00, 32.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 279    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.0%  |
| Attack success rate:          | 0.71%  |
| Average perturbed word %:     | 2.73%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.16  |
+-------------------------------+--------+


In [210]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim08.csv', target_ds_ag, 0.8, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptepsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|███▊                   | 50/300 [26:18<2:11:31, 31.56s/it]textattack: Saving checkpoint under "checkpoints\1723356440837.ta.chkpt" at 2024-08-11 14:07:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 94 / 6 / 100:  33%|███████              | 100/300 [52:51<1:45:43, 31.72s/it]textattack: Saving checkpoint under "checkpoints\1723358034228.ta.chkpt" at 2024-08-11 14:33:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 142 / 8 / 150:  50%|█████████         | 150/300 [1:20:18<1:20:18, 32.12s/it]textattack: Saving checkpoint under "checkpoints\1723359680809.ta.chkpt" at 2024-08-11 15:01:20 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 189 / 11 / 200:  67%|████████████▋      | 200/300 [1:47:14<53:37, 32.17s/it]textattack: Saving checkpoint under "checkpoints\1723361297099.ta.chkpt" at 2024-08-11 15:28:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 234 / 15 / 250:  83%|███████████████▊   | 250/300 [2:14:27<26:53, 32.27s/it]textattack: Saving checkpoint under "checkpoints\1723362930403.ta.chkpt" at 2024-08-11 15:55:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:41:20<00:00, 32.27s/it]textattack: Saving checkpoint under "checkpoints\1723364543568.ta.chkpt" at 2024-08-11 16:22:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:41:20<00:00, 32.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 280    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 2.44%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.7   |
+-------------------------------+--------+


In [211]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim09.csv', target_ds_ag, 0.9, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agbert/bertagattackptepsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|███▊                   | 50/300 [26:10<2:10:50, 31.40s/it]textattack: Saving checkpoint under "checkpoints\1723366113623.ta.chkpt" at 2024-08-11 16:48:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 94 / 6 / 100:  33%|███████              | 100/300 [52:13<1:44:26, 31.33s/it]textattack: Saving checkpoint under "checkpoints\1723367676873.ta.chkpt" at 2024-08-11 17:14:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 142 / 8 / 150:  50%|█████████         | 150/300 [1:19:13<1:19:13, 31.69s/it]textattack: Saving checkpoint under "checkpoints\1723369296714.ta.chkpt" at 2024-08-11 17:41:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 189 / 11 / 200:  67%|████████████▋      | 200/300 [1:45:58<52:59, 31.79s/it]textattack: Saving checkpoint under "checkpoints\1723370902519.ta.chkpt" at 2024-08-11 18:08:22 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 234 / 15 / 250:  83%|███████████████▊   | 250/300 [2:12:36<26:31, 31.83s/it]textattack: Saving checkpoint under "checkpoints\1723372500362.ta.chkpt" at 2024-08-11 18:35:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:39:25<00:00, 31.88s/it]textattack: Saving checkpoint under "checkpoints\1723374108720.ta.chkpt" at 2024-08-11 19:01:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:39:25<00:00, 31.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 280    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 2.44%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 29.43  |
+-------------------------------+--------+


In [ ]:
et10 = datetime.datetime.now()
et10 - et9